In [ ]:
let systemPathSeparator = System.IO.Path.DirectorySeparatorChar
let seperatorAsString = systemPathSeparator.ToString()
let stringToReplace = $"{seperatorAsString}Analysis{seperatorAsString}src{seperatorAsString}Function"
let currentProjectPath = 
        (__SOURCE_DIRECTORY__)
            .Replace(stringToReplace, "") // path name of the AnalysisTool-for-bioreactor folder


In [ ]:
currentProjectPath

/Users/simonfoellinger/Desktop/AnalysisTool-for-bioreactor

In [ ]:

type tableRow =
    {
        PhaseID : int;
        startTimeGrowphase : float;
        endTimeGrowphase : float;
        slopeOrGrowrateOfLinearRegressionOrGrowphase : float;
        duplicationTimeOfGrowphase : float
    }

type Result<'T, 'TError> =
    | Ok of 'T
    | Error of 'TError

In [ ]:

#load "/Users/simonfoellinger/Desktop/AnalysisTool-for-bioreactor/Analysis/src/Library/Library.fs"
#r "nuget: FSharp.Data"
#r "nuget: FSharp.Stats, 0.5.0"
#r "nuget: Deedle, 3.0.0"
#r "nuget: Deedle.Interactive, 3.0.0"
#r "nuget: Plotly.NET, 4.2.0"
#r "nuget: Plotly.NET.Interactive, 4.1.0"
#r "nuget: FSharpAux.IO, 2.0.0"
#r "nuget: Giraffe.ViewEngine, 1.0.0"

Installed Packages Deedle, 3.0.0 Deedle.Interactive, 3.0.0 FSharp.Data, 6.4.0 FSharp.Stats, 0.5.0 FSharpAux.IO, 2.0.0 Giraffe.ViewEngine, 1.0.0 Plotly.NET, 4.2.0 Plotly.NET.Interactive, 4.1.0

In [ ]:
// #r "nuget: FSharp.Data"
// #r "nuget: FSharp.Stats, 0.5.0"
open FSharp.Stats
open FSharp.Stats.Fitting

// #r "nuget: Deedle, 3.0.0"
// #r "nuget: Deedle.Interactive, 3.0.0"
open Deedle.Interactive
open Deedle

// #r "nuget: Plotly.NET, 4.2.0"
// #r "nuget: Plotly.NET.Interactive, 4.1.0"
open Plotly.NET.Interactive
open Plotly.NET
open Plotly.NET.LayoutObjects
open Plotly.NET.TraceObjects
open Plotly.NET.StyleParam

// #r "nuget: FSharpAux.IO, 2.0.0"
open FSharpAux.IO

// #r "nuget: Giraffe.ViewEngine, 1.0.0"
open Giraffe.ViewEngine
open GenericChart


In [ ]:


// //Analysis full with horizontal split End Graph and only 3 row table
// // let analysis (fileName:string) (upperODCut:float) (lowerODCut:float) (cylinder : int list) (result : bool)=

// // Define the type signature for `analysis` with `Exception` as the error type in `Result`.
// let analysis: string -> float -> float -> int list -> bool -> (unit * Result<unit, Exception>) = 
//     fun (fileName:string) (upperODCut:float) (lowerODCut:float) (cylinder : int list) (result : bool) ->

//         let systemPathSeparator = System.IO.Path.DirectorySeparatorChar
//         let seperatorAsString = systemPathSeparator.ToString()
//         let stringToReplace = $"{seperatorAsString}Analysis{seperatorAsString}src{seperatorAsString}Function"
//         let currentProjectPath = 
//             (__SOURCE_DIRECTORY__)
//                 .Replace(stringToReplace, "") // path name of the AnalysisTool-for-bioreactor folder
        
//         let rawData : Frame<float,string> =  
//             Frame.ReadCsv(location = (currentProjectPath + $"{seperatorAsString}InsertTableHere{seperatorAsString}" + fileName+".txt"),
//             separators = "\t", 
//             hasHeaders = true,
//             inferRows = 10000 // infer first 10000 rows for determining type
//             )
//             |> Frame.indexRows "time"

//     // function pumpData and odData
//         let getColumnsWithSubstring (frame: Frame<float,string>) (substring: string) =
//             frame.ColumnKeys
//             |> Seq.filter (fun x -> x.Contains(substring))

//     // null Array
//         let zeroArray = Array.init 250 (fun index -> (float index,0.0)) 

//         let pump_Data = 
//             let keys = getColumnsWithSubstring rawData ".pump"
//             rawData
//             |> Frame.sliceCols
//                 (keys |> Seq.sort)
//             |> Frame.mapColKeys (fun x ->
//                     match x with
//                     | _ when x.Contains("-1-") -> "pump_data_1"
//                     | _ when x.Contains("-2-") -> "pump_data_2"
//                     | _ when x.Contains("-3-") -> "pump_data_3"
//                     | _ when x.Contains("-4-") -> "pump_data_4"
//                     | _ when x.Contains("-5-") -> "pump_data_5"
//                     | _ when x.Contains("-6-") -> "pump_data_6"
//                     | _ when x.Contains("-7-") -> "pump_data_7"
//                     | _ when x.Contains("-8-") -> "pump_data_8"
//                     | _ -> x + "_Not_related_with_AnyCylinder"
//             )
//             |> Frame.fillMissing Direction.Forward

//         let light_Data = 
//             let keys = getColumnsWithSubstring rawData ".light" //|> Seq.sortWith (fun x -> x.Contains(num))
//             rawData
//             |> Frame.sliceCols
//                 (keys |> Seq.sort)
//                 |> Frame.mapColKeys (fun x -> 
//                     match x with
//                     | _ when x.Contains("-1-") -> "light_treatment_1"
//                     | _ when x.Contains("-2-") -> "light_treatment_2"
//                     | _ when x.Contains("-3-") -> "light_treatment_3"
//                     | _ when x.Contains("-4-") -> "light_treatment_4"
//                     | _ when x.Contains("-5-") -> "light_treatment_5"
//                     | _ when x.Contains("-6-") -> "light_treatment_6"
//                     | _ when x.Contains("-7-") -> "light_treatment_7"
//                     | _ when x.Contains("-8-") -> "light_treatment_8"
//                     | _ -> x
//                 )
//             |> Frame.fillMissing Direction.Forward

//         let OD680_Data = 
//             let keys = getColumnsWithSubstring rawData ".od" 
//             rawData
//             |> Frame.sliceCols
//                 (keys |> Seq.sort)
//                 |> Frame.mapColKeys (fun x -> 
//                     match x with
//                     | _ when x.Contains("-1-") -> "od_data_1"
//                     | _ when x.Contains("-2-") -> "od_data_2"
//                     | _ when x.Contains("-3-") -> "od_data_3"
//                     | _ when x.Contains("-4-") -> "od_data_4"
//                     | _ when x.Contains("-5-") -> "od_data_5"
//                     | _ when x.Contains("-6-") -> "od_data_6"
//                     | _ when x.Contains("-7-") -> "od_data_7"
//                     | _ when x.Contains("-8-") -> "od_data_8"
//                     | _ -> x
//                 )
//             |> Frame.mapValues (fun x -> log(x))

//     //Pump Data from cylinder 2,4,6,8 and all -> Series with tuple float (Time, Log OD680)
//         let pump_Data_1  = //: (float,float) array  = 
//             let pumpColumn =
//                 pump_Data.TryGetColumnObservation<float>("pump_data_1", Lookup.Exact)
//             match pumpColumn.HasValue with
//             | true -> pumpColumn.Value.Value |> Series.observations |> Seq.toArray
//             | false -> zeroArray

//         let pump_Data_2 = 
//             let pumpColumn1 =
//                 pump_Data.TryGetColumnObservation<float>("pump_data_2",Lookup.Exact)
//             match pumpColumn1.HasValue with
//             | true -> pumpColumn1.Value.Value |> Series.observations |> Seq.toArray
//             | false -> zeroArray

//         let pump_Data_3 = 
//             let pumpColumn =
//                 pump_Data.TryGetColumnObservation<float>("pump_data_3",Lookup.Exact)
//             match pumpColumn.HasValue with
//             | true -> pumpColumn.Value.Value |> Series.observations |> Seq.toArray
//             | false -> zeroArray

//         let pump_Data_4 = 
//             let pumpColumn =
//                 pump_Data.TryGetColumnObservation<float>("pump_data_4",Lookup.Exact)
//             match pumpColumn.HasValue with
//             | true -> pumpColumn.Value.Value |> Series.observations |> Seq.toArray
//             | false -> zeroArray

//         let pump_Data_5 = 
//             let pumpColumn =
//                 pump_Data.TryGetColumnObservation<float>("pump_data_5",Lookup.Exact)
//             match pumpColumn.HasValue with
//             | true -> pumpColumn.Value.Value |> Series.observations |> Seq.toArray
//             | false -> zeroArray

//         let pump_Data_6 = 
//             let pumpColumn =
//                 pump_Data.TryGetColumnObservation<float>("pump_data_6",Lookup.Exact)
//             match pumpColumn.HasValue with
//             | true -> pumpColumn.Value.Value |> Series.observations |> Seq.toArray
//             | false -> zeroArray

//         let pump_Data_7 = 
//             let pumpColumn =
//                 pump_Data.TryGetColumnObservation<float>("pump_data_7",Lookup.Exact)
//             match pumpColumn.HasValue with
//             | true -> pumpColumn.Value.Value |> Series.observations |> Seq.toArray
//             | false -> zeroArray

//         let pump_Data_8 = 
//             let pumpColumn =
//                 pump_Data.TryGetColumnObservation<float>("pump_data_8",Lookup.Exact)
//             match pumpColumn.HasValue with
//             | true -> pumpColumn.Value.Value |> Series.observations |> Seq.toArray
//             | false -> zeroArray

//         let pump_Data_all = 
//             [|pump_Data_1;pump_Data_2;pump_Data_3;pump_Data_4;pump_Data_5;pump_Data_6;pump_Data_7;pump_Data_8|]
//             |> Array.map (fun pumpArrayOfOneCylinder -> 
//                 pumpArrayOfOneCylinder
//                 |> Array.map (fun pumpvolume -> ((fst pumpvolume),((snd pumpvolume) - (snd (pumpArrayOfOneCylinder.[0])))))
//             )

//     //OD680 Data from cylinder 2,4,6,8 and all -> Array with tuple float (Time, Log OD680)
//         let OD680_Data_1 =
//             let OD_Column =
//                 OD680_Data.TryGetColumnObservation<float>("od_data_1",Lookup.Exact)
//             match OD_Column.HasValue with
//             | true -> OD_Column.Value.Value |> Series.dropMissing |> Series.observations |> Seq.toArray
//             | false -> zeroArray

//         let OD680_Data_2 = 
//             let OD_Column =
//                 OD680_Data.TryGetColumnObservation<float>("od_data_2",Lookup.Exact)
//             match OD_Column.HasValue with
//             | true -> OD_Column.Value.Value |> Series.dropMissing |> Series.observations |> Seq.toArray
//             | false -> zeroArray

//         let OD680_Data_3 = 
//             let OD_Column =
//                 OD680_Data.TryGetColumnObservation<float>("od_data_3",Lookup.Exact)
//             match OD_Column.HasValue with
//             | true -> OD_Column.Value.Value |> Series.dropMissing |> Series.observations |> Seq.toArray
//             | false -> zeroArray

//         let OD680_Data_4 = 
//             let OD_Column =
//                 OD680_Data.TryGetColumnObservation<float>("od_data_4",Lookup.Exact)
//             match OD_Column.HasValue with
//             | true -> OD_Column.Value.Value |> Series.dropMissing |> Series.observations |> Seq.toArray
//             | false -> zeroArray

//         let OD680_Data_5 = 
//             let OD_Column =
//                 OD680_Data.TryGetColumnObservation<float>("od_data_5",Lookup.Exact)
//             match OD_Column.HasValue with
//             | true -> OD_Column.Value.Value |> Series.dropMissing |> Series.observations |> Seq.toArray
//             | false -> zeroArray

//         let OD680_Data_6 = 
//             let OD_Column =
//                 OD680_Data.TryGetColumnObservation<float>("od_data_6",Lookup.Exact)
//             match OD_Column.HasValue with
//             | true -> OD_Column.Value.Value |> Series.dropMissing |> Series.observations |> Seq.toArray
//             | false -> zeroArray

//         let OD680_Data_7 = 
//             let OD_Column =
//                 OD680_Data.TryGetColumnObservation<float>("od_data_7",Lookup.Exact)
//             match OD_Column.HasValue with
//             | true -> OD_Column.Value.Value |> Series.dropMissing |> Series.observations |> Seq.toArray
//             | false -> zeroArray

//         let OD680_Data_8 = 
//             let OD_Column =
//                 OD680_Data.TryGetColumnObservation<float>("od_data_8",Lookup.Exact)
//             match OD_Column.HasValue with
//             | true -> OD_Column.Value.Value |> Series.dropMissing |> Series.observations |> Seq.toArray
//             | false -> zeroArray

//         let odData680_all = [|OD680_Data_1;OD680_Data_2;OD680_Data_3;OD680_Data_4;OD680_Data_5;OD680_Data_6;OD680_Data_7;OD680_Data_8|]

//     //light data 
//         let light_Data_1 = 
//             let lightColumn =
//                 light_Data.TryGetColumnObservation<float>("light_treatment_1",Lookup.Exact)
//             match lightColumn.HasValue with
//             | true -> lightColumn.Value.Value |> Series.observations |> Seq.toArray
//             | false -> zeroArray

//         let light_Data_2 = 
//             let lightColumn =
//                 light_Data.TryGetColumnObservation<float>("light_treatment_2",Lookup.Exact)
//             match lightColumn.HasValue with
//             | true -> lightColumn.Value.Value |> Series.observations |> Seq.toArray
//             | false -> zeroArray

//         let light_Data_3 = 
//             let lightColumn =
//                 light_Data.TryGetColumnObservation<float>("light_treatment_3",Lookup.Exact)
//             match lightColumn.HasValue with
//             | true -> lightColumn.Value.Value |> Series.observations |> Seq.toArray
//             | false -> zeroArray

//         let light_Data_4 = 
//             let lightColumn =
//                 light_Data.TryGetColumnObservation<float>("light_treatment_4",Lookup.Exact)
//             match lightColumn.HasValue with
//             | true -> lightColumn.Value.Value |> Series.observations |> Seq.toArray
//             | false -> zeroArray

//         let light_Data_5 = 
//             let lightColumn =
//                 light_Data.TryGetColumnObservation<float>("light_treatment_5",Lookup.Exact)
//             match lightColumn.HasValue with
//             | true -> lightColumn.Value.Value |> Series.observations |> Seq.toArray
//             | false -> zeroArray

//         let light_Data_6 = 
//             let lightColumn =
//                 light_Data.TryGetColumnObservation<float>("light_treatment_6",Lookup.Exact)
//             match lightColumn.HasValue with
//             | true -> lightColumn.Value.Value |> Series.observations |> Seq.toArray
//             | false -> zeroArray
                

//         let light_Data_7 = 
//             let lightColumn =
//                 light_Data.TryGetColumnObservation<float>("light_treatment_7",Lookup.Exact)
//             match lightColumn.HasValue with
//             | true -> lightColumn.Value.Value |> Series.observations |> Seq.toArray
//             | false -> zeroArray

//         let light_Data_8 = 
//             let lightColumn =
//                 light_Data.TryGetColumnObservation<float>("light_treatment_8",Lookup.Exact)
//             match lightColumn.HasValue with
//             | true -> lightColumn.Value.Value |> Series.observations |> Seq.toArray
//             | false -> zeroArray

//         let lightData_all = [|light_Data_1;light_Data_2;light_Data_3;light_Data_4;light_Data_5;light_Data_6;light_Data_7;light_Data_8|]



//     // lightphases -> array with tuple (starttime,endtime)
//         let lightphase (lightData: (float*float) []) =
//             let lightphaseTimeTupleStartEnd : (float * float) array= 
//                 lightData
//                 |> Array.groupBy (fun (timeLight,lightIntensity) -> lightIntensity)
//                 |> Array.filter (fun (constantIntensity,arrayTimeLight) -> arrayTimeLight.Length < 20000 && arrayTimeLight.Length > 300)
//                 |> Array.map (fun (constantVolume,arrayTimePumpvolume) ->
//                     (fst (arrayTimePumpvolume.[0]),fst (arrayTimePumpvolume |> Array.last)))
//             if lightphaseTimeTupleStartEnd.Length = 3 then 
//                 [|(fst lightphaseTimeTupleStartEnd.[0], snd lightphaseTimeTupleStartEnd.[1]); (fst lightphaseTimeTupleStartEnd.[2], snd lightphaseTimeTupleStartEnd.[2])|]
//             else
//                 lightphaseTimeTupleStartEnd

//         // // lightphases -> array with tuple (starttime, endtime)
//         // let lightphase (lightData: (float * float) []) =
//         //     // Group by light intensity, filtering for reasonable length constraints
//         //     let lightphaseTimeTupleStartEnd : (float * float) array = 
//         //         lightData
//         //         |> Array.groupBy (fun (timeLight, lightIntensity) -> lightIntensity)
//         //         |> Array.filter (fun (_, arrayTimeLight) -> arrayTimeLight.Length < 20000 && arrayTimeLight.Length > 300)
//         //         |> Array.map (fun (_, arrayTimePumpvolume) ->
//         //             if arrayTimePumpvolume.Length > 0 then 
//         //                 (fst arrayTimePumpvolume.[0], fst (arrayTimePumpvolume |> Array.last))
//         //             else 
//         //                 (0.0, 0.0)) // Fallback tuple in case array is unexpectedly empty

//         //     // Check the length of the result and return accordingly
//         //     match lightphaseTimeTupleStartEnd.Length with
//         //     | 3 -> 
//         //         [|
//         //             (fst lightphaseTimeTupleStartEnd.[0], snd lightphaseTimeTupleStartEnd.[1])
//         //             (fst lightphaseTimeTupleStartEnd.[2], snd lightphaseTimeTupleStartEnd.[2])
//         //         |]
//         //     | _ -> lightphaseTimeTupleStartEnd // Return the computed array if it doesn't match length 3

//         let lightphase_1 =
//             (lightphase light_Data_1)

//         let lightphase_2 =
//             (lightphase light_Data_2)

//         let lightphase_3 =
//             (lightphase light_Data_3)

//         let lightphase_4 =
//             (lightphase light_Data_4)

//         let lightphase_5 =
//             (lightphase light_Data_5)

//         let lightphase_6 =
//             (lightphase light_Data_6)

//         let lightphase_7 =
//             (lightphase light_Data_7)

//         let lightphase_8 =
//             (lightphase light_Data_8)

//         let lightphase_all = [|lightphase_1;lightphase_2;lightphase_3;lightphase_4;lightphase_5;lightphase_6;lightphase_7;lightphase_8|]

//         let sampleNumber odData =
//                 if odData = OD680_Data_1 then "1"
//                 else if odData = OD680_Data_2 then "2"
//                 else if odData = OD680_Data_3 then "3" 
//                 else if odData = OD680_Data_4 then "4" 
//                 else if odData = OD680_Data_5 then "5"
//                 else if odData = OD680_Data_6 then "6"
//                 else if odData = OD680_Data_7 then "7"  
//                 else if odData = OD680_Data_8 then "8"
//                 else "00"

//     // growphases -> array with tuple (starttime,endtime)
//         let growphase (pumpData: (float*float) array) (odData: (float*float) array) =
//             let growphaseTimeTupleStartEnd = 
//                 let arrayOfConstantPumpValue =
//                     pumpData
//                     |> Array.groupBy (fun (timePump,pumpVolume) -> pumpVolume)
//                     |> Array.filter (fun (constantVolume,arrayTimePumpvolume) -> arrayTimePumpvolume.Length > 250)
                    
//                 let startEndOfConstantPumpValue : (float * float) array =
//                     arrayOfConstantPumpValue
//                     |> Array.map (fun ((constantVolume:float),(arrayTimePumpvolume: (float * float) array)) -> 
//                         if Array.isEmpty arrayTimePumpvolume then 
//                             (0.0,0.0)
//                         else
//                             (float (fst (arrayTimePumpvolume.[0])),float (fst (arrayTimePumpvolume |> Array.last)))
//                         )
//                 startEndOfConstantPumpValue
//                 |> Array.map (fun (startOfgrowphase,endOfGrowphase) -> 
//                     odData 
//                     |> Array.filter (fun (time,od680) -> time > startOfgrowphase && time < endOfGrowphase && od680 > (log lowerODCut) && od680 < (log upperODCut) )) //evtl bei Growphase einbauen damit shapes korrekt sind
//                     |> Array.map (fun (arrayTimeOD680) ->
//                         if Array.isEmpty arrayTimeOD680 then 
//                             let res = (0.0,0.0)
//                             res
//                         else
//                         (fst (arrayTimeOD680.[0]), fst (arrayTimeOD680 |> Array.last)))
//             growphaseTimeTupleStartEnd

//         let growphase_1 =
//             (growphase pump_Data_1 OD680_Data_1)

//         let growphase_2 =
//             (growphase pump_Data_2 OD680_Data_2)
        
//         let growphase_3 =
//             (growphase pump_Data_3 OD680_Data_3)
        
//         let growphase_4 =
//             (growphase pump_Data_4 OD680_Data_4)

//         let growphase_5 =
//             (growphase pump_Data_5 OD680_Data_5)

//         let growphase_6 =
//             (growphase pump_Data_6 OD680_Data_6)
        
//         let growphase_7 =
//             (growphase pump_Data_7 OD680_Data_7)
        
//         let growphase_8 =
//             (growphase pump_Data_8 OD680_Data_8)

//         let growphase_all = [|growphase_1;growphase_2;growphase_3;growphase_4;growphase_5;growphase_6;growphase_7;growphase_8|]
    
//     // linear regression funktion 
//         let ChartGrowphase_linearFit (growphase : (float * float) array) (odData:(float * float) array) (growphase_index : int) = 
//             let arrayGrowphase_Time_OD (growphase : (float * float) array) (odData:(float * float) array) =
//                 growphase
//                 |> Array.map (fun (min,max) -> 
//                     odData 
//                     |> Array.filter (fun (time,od680) -> time > min && time < max && od680 > (log lowerODCut) && od680 < (log upperODCut) )) //evtl bei Growphase einbauen damit shapes korrekt sind

//             let growphase_time_OD (growphase_index : int)=
//                 let growphase_time_OD = (arrayGrowphase_Time_OD growphase odData).[growphase_index]
//                 growphase_time_OD

//             let xs (growphase_index : int) =
//                 let SeqOfOD680 =
//                     (growphase_time_OD growphase_index)
//                     |> Array.map (fun (time,od680) -> time)
//                 let vector_OD = vector SeqOfOD680
//                 vector_OD 

//             let ys (growphase_index : int)=
//                 let SeqOfOD680 =
//                     (growphase_time_OD growphase_index)
//                     |> Array.map (fun (time,od680) -> od680)
//                 let vector_OD = vector SeqOfOD680
//                 vector_OD 
//             let fit = 
//                 if Vector.length (xs growphase_index) > 5 && Vector.length (ys growphase_index) > 5 then
//                     Fitting.LinearRegression.fit(xs growphase_index ,ys growphase_index,FittingMethod = Fitting.Method.Robust RobustEstimator.TheilSen)
//                 else
//                     Fitting.LinearRegression.Coefficients(vector [0.0;0.0])
//             let e : float = 
//                 if Array.isEmpty (growphase_time_OD growphase_index) then
//                     0.0
//                 else
//                     (growphase_time_OD growphase_index)
//                     |> Array.map (fun (time,od680) -> time)
//                     |> Array.last
//             let s : float =
//                 if Array.isEmpty (growphase_time_OD growphase_index) then
//                     0.0
//                 else
//                     (growphase_time_OD growphase_index)
//                     |> Array.map (fun (time,od680) -> time)
//                     |> Array.sortDescending
//                     |> Array.last
//             let linaer_fit = 
//                 [s .. 0.1 .. e] 
//                 |> List.map (fun x -> x,LinearRegression.predict(fit) x)  
//                 |> Chart.Line 
//             linaer_fit


//     // shape list -> array of float tuple (starttime, Endtime)
//         let shapeGrow (growphase : (float * float) array) (odData : (float * float) array) = 
//             growphase
//             |> Array.mapi (fun i (start,finish) -> 
//                 Shape.init (
//                     ShapeType = StyleParam.ShapeType.Rectangle,
//                     X0 = start,
//                     X1 = finish,
//                     Y0 = (snd (odData |> Array.sortBy snd |> Array.last) + 0.1),
//                     Y1 = (snd (odData |> Array.sortBy snd |> Array.last) + 0.3),
//                     Opacity = 0.2,
//                     FillColor = Color.fromHex "#a3e77f",
//                     Label = ShapeLabel.init(TextTemplate = $"{i+1}", TextAngle = TextAngle.Degrees 0.0, TextPosition = TextPosition.BottomCenter )
//             )
//             )
//             |> Array.toList


//         // let shapeLight (lightphase : (float * float) array) (odData : (float * float) array) (lightdata : (float * float) array) =  
//         //     let treatment1and2Tuple =
//         //         let groupedByTreatment = 
//         //             lightdata
//         //             |> Array.groupBy (fun (time, lightData) -> lightData)
//         //             |> Array.sortByDescending (fun (lightTreatment,_) -> lightTreatment )
//         //         (fst groupedByTreatment.[1],fst groupedByTreatment.[0])

//         //     let maxYValue =
//         //         match Array.isEmpty odData with
//         //         | true -> 0.0  // or any default value you prefer
//         //         | false -> (snd (odData |> Array.sortByDescending snd |> Array.last) + 0.25)

//         //     lightphase
//         //     |> Array.mapi (fun i (start,finish) -> 
//         //             if i = 0 then 
//         //                 Shape.init (
//         //                     ShapeType = StyleParam.ShapeType.Rectangle,
//         //                     X0 = start,
//         //                     X1 = finish,
//         //                     Y0 = maxYValue,
//         //                     Y1 = 0.0, //(snd (odData |> Array.sortByDescending snd |> Array.last) + 1.0),
//         //                     Opacity = 0.1,
//         //                     FillColor = Color.fromHex "#ffffba",
//         //                     Label = ShapeLabel.init(TextTemplate = $"Highlight {fst treatment1and2Tuple}", TextAngle = TextAngle.Degrees 0.0, TextPosition = TextPosition.BottomCenter )
//         //                     )
//         //             else
//         //                 Shape.init (
//         //                     ShapeType = StyleParam.ShapeType.Rectangle,
//         //                     X0 = start,
//         //                     X1 = finish,
//         //                     Y0 = maxYValue,
//         //                     Y1 = 0.0, //(snd (odData |> Array.sortByDescending snd |> Array.last) + 1.0),
//         //                     Opacity = 0.1,
//         //                     FillColor = Color.fromHex "#ffffba",
//         //                     Label = ShapeLabel.init(TextTemplate = $"Highlight {snd treatment1and2Tuple}", TextAngle = TextAngle.Degrees 0.0, TextPosition = TextPosition.BottomCenter )
//         //                     )
//         //     )
//         //     |> Array.toList

//         let shapeLight (lightphase: (float * float) []) (odData: (float * float) []) (lightdata: (float * float) []) =  
//             if lightphase.Length = 0 || odData.Length = 0 || lightdata.Length = 0 then
//                 printfn "Warning: One or more input arrays to shapeLight are empty."
//                 [] // Return an empty list if any of the inputs are empty
//             else
//                 let treatment1and2Tuple =
//                     if lightdata.Length >= 2 then
//                         let groupedByTreatment = 
//                             lightdata
//                             |> Array.groupBy (fun (time, lightData) -> lightData)
//                             |> Array.sortByDescending (fun (lightTreatment, _) -> lightTreatment)
//                         // Check if there are at least two groups in groupedByTreatment
//                         if groupedByTreatment.Length >= 2 then
//                             (fst groupedByTreatment.[1], fst groupedByTreatment.[0])
//                         else
//                             printfn "Warning: Not enough groups in lightdata."
//                             (0.0, 0.0) // Default if there are not enough groups
//                     else
//                         printfn "Warning: lightdata has less than two elements."
//                         (0.0, 20.0) // Default if lightdata is too short

//                 let maxYValue =
//                     if Array.isEmpty odData then
//                         0.0  // Default value if odData is empty
//                     else 
//                         (snd (odData |> Array.sortByDescending snd |> Array.head) + 0.25)

//                 // Create shapes with bounds checks
//                 lightphase
//                 |> Array.mapi (fun i (start, finish) -> 
//                     let labelText = 
//                         if i = 0 then $"Highlight {fst treatment1and2Tuple}"
//                         else $"Highlight {snd treatment1and2Tuple}"
                    
//                     Shape.init (
//                         ShapeType = StyleParam.ShapeType.Rectangle,
//                         X0 = start,
//                         X1 = finish,
//                         Y0 = maxYValue,
//                         Y1 = 0.0,
//                         Opacity = 0.1,
//                         FillColor = Color.fromHex "#ffffba",
//                         Label = ShapeLabel.init(TextTemplate = labelText, TextAngle = TextAngle.Degrees 0.0, TextPosition = TextPosition.BottomCenter)
//                     )
//                 )
//                 |> Array.toList

//     // pump-graph combined with OD graph and linear fit for growphase
//         let endGraph (growphase : (float * float) array) (odData:(float * float) array) (pump_Data : (float * float) array) (lightData : (float * float) array)=
//             let linearRegression_all_chartLine =
//                 [
//                     for i in 0 .. growphase.Length - 1 do
//                     (ChartGrowphase_linearFit growphase odData i) |> Chart.withTraceInfo $"Robust TheilSen {i}"
//                     |> Chart.withLineStyle(Width = 2.25, Color = Color.fromHex("#fb2e01"), Dash=StyleParam.DrawingStyle.Dot)
//                     |> Chart.withAxisAnchor(X=1,Y=1)
//                     |> Chart.withTraceInfo(Name = $"Phase ID: {i}")
//                 ]
//                 |> Chart.combine
//             let odData_all_chartPoint =
//                 Chart.Point(xy = (odData))
//                 |> Chart.withLineStyle(Color =Color.fromHex("#7bc043"))
//                 |> Chart.withTraceInfo(Name = "Ln OD680")
//                 |> Chart.withAxisAnchor(X=1,Y=1)
//             let pumpData_all_chartPoint = 
//                 Chart.Point(xy = (pump_Data))
//                 |> Chart.withLineStyle(Color =Color.fromHex("#005b96"))
//                 |> Chart.withTraceInfo(Name = "pumpvolume (ml)")
//                 |> Chart.withAxisAnchor(X=1,Y=2)
//             let lightData_all_chartPoint = 
//                 Chart.Point(xy = (lightData))
//                 |> Chart.withLineStyle(Color =Color.fromHex("#ffffba"))
//                 |> Chart.withTraceInfo(Name = "Lighttreatment (light?)")
//                 |> Chart.withAxisAnchor(X=1,Y=2)
//             [
//                 odData_all_chartPoint
//                 linearRegression_all_chartLine
//                 lightData_all_chartPoint
//                 pumpData_all_chartPoint
//             ]
//             |> Chart.combine
//             |> Chart.withXAxisStyle("time (h)",
//                 Id=StyleParam.SubPlotId.XAxis 1,
//                 MinMax= (0.0, (fst (odData |> Array.last)) + 1.0),
//                 Side=StyleParam.Side.Bottom,
//                 ShowLine=true,
//                 LineColor=Color.fromKeyword Black
//                 )
//             |> Chart.withYAxisStyle(
//                 $"Ln OD680 cylinder {sampleNumber odData}",
//                 Side=StyleParam.Side.Left,
//                 MinMax= (((snd (odData |> Array.sortByDescending snd |> Array.last)) + 0.25),((snd (odData |> Array.sortBy snd |> Array.last)) + 0.2)),
//                 Id=StyleParam.SubPlotId.YAxis 1,
//                 ShowLine=true,
//                 LineColor=Color.fromKeyword Black
//                 )
//             |> Chart.withYAxisStyle(
//                 $"pumpvolume (ml) and light treatment (µE) cylinder {sampleNumber odData}", 
//                 Side=StyleParam.Side.Right,
//                 MinMax= (((snd (lightData |> Array.sortByDescending snd |> Array.last)) - 0.0),((snd (lightData |> Array.sortBy snd |> Array.last)) * 1.5)),
//                 Id=StyleParam.SubPlotId.YAxis 2,
//                 ShowLine=true,
//                 LineColor= Color.fromKeyword(Black)
//                 )
//         printfn("Bis Hier")
//     // pump-graph combined with OD graph and linear fit for growphase
//         let endGraph_single (growphase : (float * float) array) (lightphase : (float * float) array) (odData:(float * float) array) (pump_Data : (float * float) array) (lightData : (float * float) array) =
//             let linearRegression_all_chartLine =
//                 [
//                     for i in 0 .. growphase.Length - 1 do
//                     (ChartGrowphase_linearFit growphase odData i) |> Chart.withTraceInfo $"Robust TheilSen {i}"
//                     |> Chart.withLineStyle(Width = 2.25, Color = Color.fromHex("#fb2e01"), Dash=StyleParam.DrawingStyle.Dot)
//                     |> Chart.withAxisAnchor(Y=1)
//                     |> Chart.withTraceInfo(Name = $"Phase ID: {i}")
//                 ]
//                 |> Chart.combine
//             let odData_all_chartPoint =
//                 Chart.Point(xy = (odData))
//                 |> Chart.withLineStyle(Color =Color.fromHex("#7bc043"))
//                 |> Chart.withTraceInfo(Name = "Ln OD680")
//                 |> Chart.withAxisAnchor(Y=1)
//             let pumpData_all_chartPoint = 
//                 Chart.Point(xy = (pump_Data))
//                 |> Chart.withLineStyle(Color =Color.fromHex("#005b96"))
//                 |> Chart.withTraceInfo(Name = "pumpvolume (ml)")
//                 |> Chart.withAxisAnchor(Y=2)
//             let lightData_all_chartPoint = 
//                 Chart.Point(xy = (lightData))
//                 |> Chart.withLineStyle(Color =Color.fromHex("#ffffba"))
//                 |> Chart.withTraceInfo(Name = "Lighttreatment (µE)")
//                 |> Chart.withAxisAnchor(Y=2)
//             [
//                 odData_all_chartPoint
//                 linearRegression_all_chartLine
//                 lightData_all_chartPoint
//                 pumpData_all_chartPoint
//             ]
//             |> Chart.combine
//             |> Chart.withShapes(shapes = (shapeGrow growphase odData), Append = true)
//             |> Chart.withShapes(shapes = (shapeLight lightphase odData lightData), Append = true)
//             |> Chart.withTemplate ChartTemplates.lightMirrored
//             |> Chart.withLegendStyle(Orientation = StyleParam.Orientation.Horizontal)
//             |> Chart.withSize(Width = 1600, Height = 800)
//             |> Chart.withTitle($"OD680 with linear regression of the growphases of cylinder {sampleNumber odData} with pumpdata")
//             |> Chart.withXAxisStyle("time (h)",
//                 Id=StyleParam.SubPlotId.XAxis 1,
//                 MinMax= (0.0, (fst (odData |> Array.last)) + 1.0),
//                 Side=StyleParam.Side.Bottom,
//                 ShowLine=true,
//                 LineColor=Color.fromKeyword Black
//                 )
//             |> Chart.withYAxisStyle(
//                 $"Ln OD680 cylinder {sampleNumber odData}",
//                 Side=StyleParam.Side.Left,
//                 MinMax= (((snd (odData |> Array.sortByDescending snd |> Array.last)) + 0.25),((snd (odData |> Array.sortBy snd |> Array.last)) + 0.2)),
//                 Id=StyleParam.SubPlotId.YAxis 1,
//                 Overlaying= StyleParam.LinearAxisId.Y 2,
//                 ShowLine=true,
//                 LineColor=Color.fromKeyword Black
//                 )
//             |> Chart.withYAxisStyle(
//                 $"pumpvolume (ml) and light treatment (µE) cylinder {sampleNumber odData}", 
//                 Side=StyleParam.Side.Right,
//                 MinMax= (((snd (lightData |> Array.sortByDescending snd |> Array.last)) - 0.0),((snd (lightData |> Array.sortBy snd |> Array.last)) * 1.5)),
//                 Id=StyleParam.SubPlotId.YAxis 2,
//                 ShowLine=false,
//                 LineColor= Color.fromKeyword(Black)
//                 )

//         printfn("Bis Hier 1")
//         // printfn(shapeLight lightphase odData lightData)
//     // slope/growratefunction for individual growphase of one cylinder
//         let slopeOfGrowphase_X (growphase : (float * float) array) (odData:(float * float) array) (growphase_index : int) = 
//             let arrayGrowphase_Time_OD (growphase : (float * float) array) (odData:(float * float) array) =
//                 growphase
//                 |> Array.map (fun (min,max) -> 
//                     odData 
//                     |> Array.filter (fun (time,od680) -> time > min && time < max && od680 > (log lowerODCut) && od680 < (log upperODCut)))

//             let growphase_time_OD (growphase_index : int)=
//                 let growphase_time_OD = (arrayGrowphase_Time_OD growphase odData).[growphase_index]
//                 growphase_time_OD

//             let xs (growphase_index : int) =
//                 let SeqOfOD680 =
//                     (growphase_time_OD growphase_index)
//                     |> Array.map (fun (time,od680) -> time)
//                 let vector_OD = vector SeqOfOD680
//                 vector_OD 

//             let ys (growphase_index : int)=
//                 let SeqOfOD680 =
//                     (growphase_time_OD growphase_index)
//                     |> Array.map (fun (time,od680) -> od680)
//                 let vector_OD = vector SeqOfOD680
//                 vector_OD 
//             let fit = 
//                 if Vector.length (xs growphase_index) > 5 && Vector.length (ys growphase_index) > 5 then
//                     Fitting.LinearRegression.fit(xs growphase_index ,ys growphase_index,FittingMethod = Fitting.Method.Robust RobustEstimator.TheilSen)
//                 else
//                     Fitting.LinearRegression.Coefficients(vector [0.0;0.0])
//             fit.Coefficients.[1]
//         printfn("Bis Hier2")
//     // function for table values -> List List string
//         let table_list (growphase : (float * float) array) (odData:(float * float) array) : list<Library.tableRow> = 
//             let growrateList =
//                 [
//                     for i in 0 .. growphase.Length - 1 do
//                     (slopeOfGrowphase_X growphase odData i)
//                 ]
//                 |> List.toArray
//             let list_int = 
//                 [|1 .. growphase.Length|]
//             let start = 
//                 growphase
//                 |> Array.map (fun (start,finish) -> start)
//             let finish = 
//                 growphase
//                 |> Array.map (fun (start,finish) -> finish)
//             let rowItemsList : Library.tableRow list =
//                 [for i in 0 .. growphase.Length - 1 do
//                     let list_int_index: int = list_int.[i]
//                     let start_index: float = start.[i]
//                     let finish_index: float = finish.[i]
//                     let growrate_index: float = growrateList.[i]
//                     let duplicationTime: float = (log(2.0) /(growrate_index))
//                     {
//                         Library.tableRow.PhaseID = list_int_index;
//                         Library.tableRow.startTimeGrowphase = start_index;
//                         Library.tableRow.endTimeGrowphase = finish_index;
//                         Library.tableRow.slopeOrGrowrateOfLinearRegressionOrGrowphase = growrate_index;
//                         Library.tableRow.duplicationTimeOfGrowphase = duplicationTime
//                     }
//                 ]
//             rowItemsList
//         printfn("Bis Hier3")
//     // function for boxblot of slopes of one cylinder
//         let boxplot_slope (growphase : (float * float) array) (odData:(float * float) array) = 
//             let y = 
//                 table_list (growphase) (odData) |> List.map (fun x -> float x.slopeOrGrowrateOfLinearRegressionOrGrowphase)
//             Chart.BoxPlot(Y = y, Name = "growrate (h<sup>-1</sub>)",Jitter=0.1,BoxPoints=StyleParam.BoxPoints.SuspectedOutliers,BoxMean=StyleParam.BoxMean.True, OutlineWidth = 1.5)
//             |> Chart.withLineStyle(Color = Color.fromHex("#005b96"))

//     // function for pointchart of slopes of one cylinder
//         let pointchart_slope (growphase : (float * float) array) (odData:(float * float) array) = 
//             let xy = 
//                 table_list (growphase) (odData) |> Seq.map (fun x -> (float x.PhaseID, float x.slopeOrGrowrateOfLinearRegressionOrGrowphase)) 
//             Chart.Point(xy = xy, Name = "growrate (h<sup>-1</sub>)")
//             |> Chart.withLineStyle(Color = Color.fromHex("#005b96"))

//         let sortSlopeList (growphase : (float * float) array) (odData:(float * float) array) = 
//             let slopelist = 
//                 (table_list growphase odData)
//                 |> List.map (fun rowItem -> (rowItem.slopeOrGrowrateOfLinearRegressionOrGrowphase))
//             slopelist
//             |> List.sort
//         printfn("Bis Hier4")
//     // function for table
//         let table (growphase : (float * float) array) (odData:(float * float) array) = 
//             let header : seq<string> = seq [ "<b>Phase ID</b>"; "starttime of growphase (h)"; "endtime of growphase (h)"; "growrate (h<sup>-1</sub>)"; "duplicationtime (Td) (h)" ]
//             let rows : seq<seq<float>> = 
//                 table_list growphase odData
//                 |> Seq.map (fun rowItem ->
//                         seq [
//                             float rowItem.PhaseID;
//                             round 2 rowItem.startTimeGrowphase;
//                             round 2 rowItem.endTimeGrowphase;
//                             round 3 rowItem.slopeOrGrowrateOfLinearRegressionOrGrowphase;
//                             round 3 rowItem.duplicationTimeOfGrowphase
//                             ]
//                     )
//             Chart.Table(
//                 header,
//                 rows,
//                 HeaderAlign = StyleParam.HorizontalAlign.Left,
//                 CellsAlign = StyleParam.HorizontalAlign.Left,
//                 MultiColumnWidth = [1.5;2.5;2.5;4.0],
//                 HeaderFillColor = Color.fromString( "DarkGray" ),
//                 HeaderHeight = 2,
//                 HeaderOutlineColor = Color.fromString( "Grey" ),
//                 HeaderOutlineWidth = 2.0,
//                 CellsFillColor = Color.fromString( "LightGray" ),
//                 CellsOutlineColor = Color.fromString( "Grey" ),
//                 CellsOutlineWidth = 1.5,
//                 CellsHeight = 25
//                 )
//             |> Chart.withSize(Width=1600,Height=800)

//     // function for table
//         // let description (growphase : (float * float) array) (odData:(float * float) array) = 
//         //     // some styling for a html table
//         //     let style = "<style>table {font-family: arial, sans-serif;border-collapse: collapse;width: 80%;height: 100%}td, th {border: 2px solid #dddddd;text-align: left;padding: 8px;}tr:nth-child(even) {background-color: #dddddd;}</style>"
//         //     // header row of the table
//         //     let header = "<tr><th>Phase ID</th><th>starttime of growphase (h)</th><th>endtime of growphase (h)</th><th>growrate (h<sup>-1</sup>)</th><th>duplicationtime (h)</th></tr>"
//         //     // table rows
//         //     let rows = 
//         //         (table_list growphase odData)
//         //         |> List.map (fun rowItem ->
//         //                 // create a table row with phase id, the start and end of the treatment formatted as a float with one significant figure (defined by %.1f) and the slope with four significant figures.
//         //                 // the slope is stored within the fit coefficients as [intersect;slope]
//         //             $"<tr><td>{rowItem.PhaseID}</td><td>%.2f{(rowItem.startTimeGrowphase)}</td><td>%.2f{(rowItem.endTimeGrowphase)}</td><td>%.4f{(rowItem.slopeOrGrowrateOfLinearRegressionOrGrowphase)}</td><td>%.4f{(rowItem.duplicationTimeOfGrowphase)}</td></tr>"
//         //         )
//         //     // constructed table
//         //     let table = $"{style}<table>{header}{rows}</table>"
//         //     // convert the table string to a giraffe node element to be compatible with Plotly.NET
//         //     Giraffe.ViewEngine.HtmlElements.rawText table

//     // analysis function 
//         let analysisFull (growphase : (float * float) array) (lightphase : (float * float) array) (odData:(float * float) array) (pump_Data :(float * float) array) (lightData : (float * float) array) =

//             let subplotGrid = 
//                 [|
//                     [| StyleParam.LinearAxisId.X 1 , StyleParam.LinearAxisId.Y 1 ; StyleParam.LinearAxisId.X 2 , StyleParam.LinearAxisId.Y 2 |]
//                     [| StyleParam.LinearAxisId.X 3 , StyleParam.LinearAxisId.Y 3 ; StyleParam.LinearAxisId.X 4 , StyleParam.LinearAxisId.Y 4 |]
//                 |]

//             let pumpData_all_chartPoint = 
//                 Chart.Point(xy = (pump_Data))
//                 |> Chart.withLineStyle(Color =Color.fromHex("#005b96"))
//                 |> Chart.withTraceInfo(Name = "pumpvolume (ml)")

//             [
                
//                 (endGraph (growphase) (odData) (pump_Data) (lightData))
//                 |> Chart.withTemplate ChartTemplates.lightMirrored
//                 |> Chart.withShapes(shapes = (shapeGrow growphase odData), Append = true)
//                 |> Chart.withAxisAnchor(X=1,Y=1);

//                 //(endGraph (growphase) (odData) (pump_Data))
//                 pumpData_all_chartPoint
//                 |> Chart.withTemplate ChartTemplates.lightMirrored
//                 //|> Chart.withLayoutStyle(PlotBGColor = Color.fromARGB 0 0 0 0, PaperBGColor = Color.fromARGB 0 0 0 0)
//                 |> Chart.withAxisAnchor(X=1,Y=2);
            
//                 (pointchart_slope (growphase) (odData))
//                 |> Chart.withTemplate ChartTemplates.lightMirrored
//                 |> Chart.withAxisAnchor(X=3,Y=3);

//                 (boxplot_slope (growphase) (odData))
//                 |> Chart.withTemplate ChartTemplates.lightMirrored
//                 |> Chart.withAxisAnchor(X=4,Y=4);

//                 table growphase odData
//                 |> Chart.withTemplate ChartTemplates.lightMirrored
//                 //|> Chart.withAxisAnchor(X=5,Y=5);

//             ]
//             |> Chart.Grid(nRows=3, nCols=2, Pattern = StyleParam.LayoutGridPattern.Independent)
//             |> Chart.withTemplate ChartTemplates.lightMirrored
//             |> Chart.withShapes(shapes = (shapeGrow growphase odData), Append = true)
//             |> Chart.withShapes(shapes = (shapeLight lightphase odData lightData), Append = true)
//             |> Chart.withLegendStyle(Orientation = StyleParam.Orientation.Horizontal)
//             |> Chart.withXAxisStyle("",
//                 Id=StyleParam.SubPlotId.XAxis 1,
//                 MinMax= (0.0, (fst (odData |> Array.last)) + 1.0),
//                 Domain = (0.00, 1.00),
//                 Side=StyleParam.Side.Bottom,
//                 ShowLine=true,
//                 LineColor=Color.fromKeyword Black
//                 )
//             |> Chart.withYAxisStyle(
//                 $"Ln OD680 cylinder {sampleNumber odData}",
//                 Side=StyleParam.Side.TopLeft,
//                 MinMax= (((snd (odData |> Array.sortByDescending snd |> Array.last)) + 0.25 ),((snd (odData |> Array.sortBy snd |> Array.last)) + 0.2)),
//                 Id=StyleParam.SubPlotId.YAxis 1,
//                 Domain = (0.725, 1.00),
//                 ShowLine=true,
//                 LineColor=Color.fromKeyword Black
//                 )
//             |> Chart.withYAxisStyle(
//                 $"pumpvolume (ml) cylinder {sampleNumber odData}", 
//                 Side=StyleParam.Side.Right,
//                 MinMax= (((snd (pump_Data |> Array.sortByDescending snd |> Array.last)) - 0.05),((snd (pump_Data |> Array.sortBy snd |> Array.last)) * 1.)),
//                 Id=StyleParam.SubPlotId.YAxis 2,
//                 //Overlaying=StyleParam.LinearAxisId.Y 1,
//                 Domain = (0.625, 0.725),
//                 ShowLine=true,
//                 LineColor=Color.fromKeyword Black
//                 )
//             |> Chart.withXAxisStyle("time (h)",
//                 Id=StyleParam.SubPlotId.XAxis 2,
//                 MinMax= (0.0, (fst (odData |> Array.last)) + 1.0),
//                 Domain = (0.00, 1.00),
//                 Side=StyleParam.Side.Bottom,
//                 ShowLine=true,
//                 LineColor=Color.fromKeyword Black
//                 )
//             |> Chart.withXAxisStyle($"growphases in cylinder {sampleNumber odData}",
//                 Id=StyleParam.SubPlotId.XAxis 3,
//                 MinMax= (0.0, growphase.Length + 1),
//                 Domain = (0.00, 0.49),
//                 Side=StyleParam.Side.Bottom,
//                 ShowLine=true,
//                 LineColor=Color.fromKeyword Black
//                 )
//             |> Chart.withYAxisStyle("growrate (h<sup>-1</sub>)",
//                 Id=StyleParam.SubPlotId.YAxis 3,
//                 MinMax= (((sortSlopeList growphase odData).[0]) - 0.005, ((sortSlopeList growphase odData) |> List.last) + 0.005),
//                 Domain = (0.325, 0.55),
//                 Side=StyleParam.Side.Left,
//                 ShowLine=true,
//                 LineColor=Color.fromKeyword Black
//                 )
//             |> Chart.withXAxisStyle($"cylinder {sampleNumber odData}",
//                 Id=StyleParam.SubPlotId.XAxis 4,
//                 Domain = (0.51, 1.00),
//                 Side=StyleParam.Side.Bottom,
//                 ShowLine=true,
//                 LineColor=Color.fromKeyword Black
//                 )      
//             |> Chart.withYAxisStyle("growrate (h<sup>-1</sub>)",
//                 Id=StyleParam.SubPlotId.YAxis 4,
//                 MinMax= (((sortSlopeList growphase odData).[0]) - 0.005, ((sortSlopeList growphase odData) |> List.last) + 0.005),
//                 Domain = (0.325, 0.55),
//                 Side=StyleParam.Side.Right,
//                 ShowLine=true,
//                 LineColor=Color.fromKeyword Black
//                 )
//             |> Chart.withXAxisStyle("",
//                 Id=StyleParam.SubPlotId.XAxis 5,
//                 Domain = (0.00, 0.33),
//                 Side=StyleParam.Side.Bottom,
//                 ShowLine=true,
//                 LineColor=Color.fromKeyword Black
//                 )
//             |> Chart.withYAxisStyle(
//                 "",
//                 Side=StyleParam.Side.TopLeft,
//                 Id=StyleParam.SubPlotId.YAxis 5,
//                 Domain = (0.00, 1.00),
//                 ShowLine=true,
//                 LineColor=Color.fromKeyword Black
//                 )
//             |> Chart.withLayoutGridStyle (YGap = 0.3, XGap = 0.1, SubPlots = subplotGrid)
//             |> Chart.withLayoutStyle(ShowLegend = false)
//             |> Chart.withLayoutStyle(Font=(Font.init(Family= StyleParam.FontFamily.Arial,Size=14)))
//             |> Chart.withSize (1500, 1600)
//             |> Chart.withConfig (Config.init (ToImageButtonOptions = ConfigObjects.ToImageButtonOptions.init(Format = StyleParam.ImageFormat.SVG)))
//             //|> Chart.withDescription [description growphase odData]
//             |> Chart.withTitle($"growphases analysis of cylinder {sampleNumber odData}")
//         let tableExport (growphase : (float * float) array) (odData:(float * float) array) =
//             FileIO.writeToFile true (currentProjectPath + $"{seperatorAsString}Output{seperatorAsString}" + $"tableOfCylinder_{sampleNumber odData}" + ".csv") ( (table_list growphase odData) |> Seq.map (fun rowItem -> $"{rowItem.PhaseID};%.2f{(rowItem.startTimeGrowphase)};%.2f{(rowItem.endTimeGrowphase)};%.4f{(rowItem.slopeOrGrowrateOfLinearRegressionOrGrowphase)};%.4f{(rowItem.duplicationTimeOfGrowphase)}"))
//         let tableshow (growphase : (float * float) array) (odData:(float * float) array) =
//             table growphase odData
//             |> Chart.withTitle($"table: growphase characteristics of cylinder {sampleNumber odData}")
//     //do analysisFull for all 8 cylinder
//         // for i in cylinder do
//         //     //(lightphase(lightData_all.[i])) |> Array.iter (fun (x,y) -> printf $"{x},{y}")
//         //     // (growphase(pump_Data_all.[i],odData680_all.[i])) |> Array.iter (fun (x,y) -> printf $"{x},{y}")
//         //     // pump_Data_all.[i] |> Array.iter (fun (x,y) -> printf $"{x},{y}")
//         //     odData680_all.[i] |> Array.iter (fun (x,y) -> printf $"{x},{y}")
//         if result then 
//             ()
//             ,Ok()
//         else
//             for i in cylinder do
//                 (tableExport growphase_all.[i] odData680_all.[i])
//                 (endGraph_single growphase_all.[i] lightphase_all.[i] odData680_all.[i] pump_Data_all.[i] lightData_all.[i])
//                 |> Chart.saveHtml(path = (currentProjectPath + $"{seperatorAsString}Output{seperatorAsString}" + $"simpleAnalysisOfCylinder_{(sampleNumber odData680_all.[i])}" + ".html"), OpenInBrowser = false )
//                 //(description growphase_all.[i] odData680_all.[i]) |> Giraffe.ViewEngine.RenderView.AsString.xmlNode;
//                 (analysisFull growphase_all.[i] lightphase_all.[i] odData680_all.[i] pump_Data_all.[i] lightData_all.[i]) 
//                 |> Chart.saveHtml(path = (currentProjectPath + $"{seperatorAsString}Output{seperatorAsString}" + $"advancedAnalysisOfCylinder_{(sampleNumber odData680_all.[i])}" + ".html"), OpenInBrowser = false )
//                 (analysisFull growphase_all.[i] lightphase_all.[i] odData680_all.[i] pump_Data_all.[i] lightData_all.[i]) 
//                 |> Chart.show
//                 (endGraph_single growphase_all.[i] lightphase_all.[i] odData680_all.[i] pump_Data_all.[i] lightData_all.[i])
//                 |> Chart.show
//             ,Ok()
    

In [ ]:
// let (output, outcome) = analysis "20241007_DK_CBB_TS_Run5_First19hrs_ProcessedDetetedDouble" 50 0.4 [0..7] false
// printfn "Output: %A" output
// printfn "Outcome: %A" outcome

In [ ]:
let fileName = "20241007_DK_CBB_TS_Run5_First19hrs_ProcessedDetetedDouble"
let upperODCut = 50
let lowerODCut = 0.0

In [ ]:
let systemPathSeparator = System.IO.Path.DirectorySeparatorChar
let seperatorAsString = systemPathSeparator.ToString()
let stringToReplace = $"{seperatorAsString}Analysis{seperatorAsString}src{seperatorAsString}Function"
let currentProjectPath = 
    (__SOURCE_DIRECTORY__)
        .Replace(stringToReplace, "") // path name of the AnalysisTool-for-bioreactor folder

let rawData : Frame<float,string> =  
    Frame.ReadCsv(location = (currentProjectPath + $"{seperatorAsString}InsertTableHere{seperatorAsString}" + fileName+".txt"),
    separators = "\t", 
    hasHeaders = true,
    inferRows = 10000 // infer first 10000 rows for determining type
    )
    |> Frame.indexRows "time"

// function pumpData and odData
let getColumnsWithSubstring (frame: Frame<float,string>) (substring: string) =
    frame.ColumnKeys
    |> Seq.filter (fun x -> x.Contains(substring))

// null Array
let zeroArray = Array.init 250 (fun index -> (float index,0.0)) 

In [ ]:

let pump_Data = 
    let keys = getColumnsWithSubstring rawData ".pump"
    rawData
    |> Frame.sliceCols
        (keys |> Seq.sort)
    |> Frame.mapColKeys (fun x ->
            match x with
            | _ when x.Contains("-1-") -> "pump_data_1"
            | _ when x.Contains("-2-") -> "pump_data_2"
            | _ when x.Contains("-3-") -> "pump_data_3"
            | _ when x.Contains("-4-") -> "pump_data_4"
            | _ when x.Contains("-5-") -> "pump_data_5"
            | _ when x.Contains("-6-") -> "pump_data_6"
            | _ when x.Contains("-7-") -> "pump_data_7"
            | _ when x.Contains("-8-") -> "pump_data_8"
            | _ -> x + "_Not_related_with_AnyCylinder"
    )
    |> Frame.fillMissing Direction.Forward

let light_Data = 
    let keys = getColumnsWithSubstring rawData ".light" //|> Seq.sortWith (fun x -> x.Contains(num))
    rawData
    |> Frame.sliceCols
        (keys |> Seq.sort)
        |> Frame.mapColKeys (fun x -> 
            match x with
            | _ when x.Contains("-1-") -> "light_treatment_1"
            | _ when x.Contains("-2-") -> "light_treatment_2"
            | _ when x.Contains("-3-") -> "light_treatment_3"
            | _ when x.Contains("-4-") -> "light_treatment_4"
            | _ when x.Contains("-5-") -> "light_treatment_5"
            | _ when x.Contains("-6-") -> "light_treatment_6"
            | _ when x.Contains("-7-") -> "light_treatment_7"
            | _ when x.Contains("-8-") -> "light_treatment_8"
            | _ -> x
        )
    |> Frame.fillMissing Direction.Forward

let OD680_Data = 
    let keys = getColumnsWithSubstring rawData ".od" 
    rawData
    |> Frame.sliceCols
        (keys |> Seq.sort)
        |> Frame.mapColKeys (fun x -> 
            match x with
            | _ when x.Contains("-1-") -> "od_data_1"
            | _ when x.Contains("-2-") -> "od_data_2"
            | _ when x.Contains("-3-") -> "od_data_3"
            | _ when x.Contains("-4-") -> "od_data_4"
            | _ when x.Contains("-5-") -> "od_data_5"
            | _ when x.Contains("-6-") -> "od_data_6"
            | _ when x.Contains("-7-") -> "od_data_7"
            | _ when x.Contains("-8-") -> "od_data_8"
            | _ -> x
        )
    |> Frame.mapValues (fun x -> log(x))

In [ ]:

//Pump Data from cylinder 2,4,6,8 and all -> Series with tuple float (Time, Log OD680)
let pump_Data_1  = //: (float,float) array  = 
    let pumpColumn =
        pump_Data.TryGetColumnObservation<float>("pump_data_1", Lookup.Exact)
    match pumpColumn.HasValue with
    | true -> pumpColumn.Value.Value |> Series.observations |> Seq.toArray
    | false -> zeroArray

let pump_Data_2 = 
    let pumpColumn1 =
        pump_Data.TryGetColumnObservation<float>("pump_data_2",Lookup.Exact)
    match pumpColumn1.HasValue with
    | true -> pumpColumn1.Value.Value |> Series.observations |> Seq.toArray
    | false -> zeroArray

let pump_Data_3 = 
    let pumpColumn =
        pump_Data.TryGetColumnObservation<float>("pump_data_3",Lookup.Exact)
    match pumpColumn.HasValue with
    | true -> pumpColumn.Value.Value |> Series.observations |> Seq.toArray
    | false -> zeroArray

let pump_Data_4 = 
    let pumpColumn =
        pump_Data.TryGetColumnObservation<float>("pump_data_4",Lookup.Exact)
    match pumpColumn.HasValue with
    | true -> pumpColumn.Value.Value |> Series.observations |> Seq.toArray
    | false -> zeroArray

let pump_Data_5 = 
    let pumpColumn =
        pump_Data.TryGetColumnObservation<float>("pump_data_5",Lookup.Exact)
    match pumpColumn.HasValue with
    | true -> pumpColumn.Value.Value |> Series.observations |> Seq.toArray
    | false -> zeroArray

let pump_Data_6 = 
    let pumpColumn =
        pump_Data.TryGetColumnObservation<float>("pump_data_6",Lookup.Exact)
    match pumpColumn.HasValue with
    | true -> pumpColumn.Value.Value |> Series.observations |> Seq.toArray
    | false -> zeroArray

let pump_Data_7 = 
    let pumpColumn =
        pump_Data.TryGetColumnObservation<float>("pump_data_7",Lookup.Exact)
    match pumpColumn.HasValue with
    | true -> pumpColumn.Value.Value |> Series.observations |> Seq.toArray
    | false -> zeroArray

let pump_Data_8 = 
    let pumpColumn =
        pump_Data.TryGetColumnObservation<float>("pump_data_8",Lookup.Exact)
    match pumpColumn.HasValue with
    | true -> pumpColumn.Value.Value |> Series.observations |> Seq.toArray
    | false -> zeroArray

let pump_Data_all = 
    [|pump_Data_1;pump_Data_2;pump_Data_3;pump_Data_4;pump_Data_5;pump_Data_6;pump_Data_7;pump_Data_8|]
    |> Array.map (fun pumpArrayOfOneCylinder -> 
        pumpArrayOfOneCylinder
        |> Array.map (fun pumpvolume -> ((fst pumpvolume),((snd pumpvolume) - (snd (pumpArrayOfOneCylinder.[0])))))
    )

//OD680 Data from cylinder 2,4,6,8 and all -> Array with tuple float (Time, Log OD680)
let OD680_Data_1 =
    let OD_Column =
        OD680_Data.TryGetColumnObservation<float>("od_data_1",Lookup.Exact)
    match OD_Column.HasValue with
    | true -> OD_Column.Value.Value |> Series.dropMissing |> Series.observations |> Seq.toArray
    | false -> zeroArray

let OD680_Data_2 = 
    let OD_Column =
        OD680_Data.TryGetColumnObservation<float>("od_data_2",Lookup.Exact)
    match OD_Column.HasValue with
    | true -> OD_Column.Value.Value |> Series.dropMissing |> Series.observations |> Seq.toArray
    | false -> zeroArray

let OD680_Data_3 = 
    let OD_Column =
        OD680_Data.TryGetColumnObservation<float>("od_data_3",Lookup.Exact)
    match OD_Column.HasValue with
    | true -> OD_Column.Value.Value |> Series.dropMissing |> Series.observations |> Seq.toArray
    | false -> zeroArray

let OD680_Data_4 = 
    let OD_Column =
        OD680_Data.TryGetColumnObservation<float>("od_data_4",Lookup.Exact)
    match OD_Column.HasValue with
    | true -> OD_Column.Value.Value |> Series.dropMissing |> Series.observations |> Seq.toArray
    | false -> zeroArray

let OD680_Data_5 = 
    let OD_Column =
        OD680_Data.TryGetColumnObservation<float>("od_data_5",Lookup.Exact)
    match OD_Column.HasValue with
    | true -> OD_Column.Value.Value |> Series.dropMissing |> Series.observations |> Seq.toArray
    | false -> zeroArray

let OD680_Data_6 = 
    let OD_Column =
        OD680_Data.TryGetColumnObservation<float>("od_data_6",Lookup.Exact)
    match OD_Column.HasValue with
    | true -> OD_Column.Value.Value |> Series.dropMissing |> Series.observations |> Seq.toArray
    | false -> zeroArray

let OD680_Data_7 = 
    let OD_Column =
        OD680_Data.TryGetColumnObservation<float>("od_data_7",Lookup.Exact)
    match OD_Column.HasValue with
    | true -> OD_Column.Value.Value |> Series.dropMissing |> Series.observations |> Seq.toArray
    | false -> zeroArray

let OD680_Data_8 = 
    let OD_Column =
        OD680_Data.TryGetColumnObservation<float>("od_data_8",Lookup.Exact)
    match OD_Column.HasValue with
    | true -> OD_Column.Value.Value |> Series.dropMissing |> Series.observations |> Seq.toArray
    | false -> zeroArray

let odData680_all = [|OD680_Data_1;OD680_Data_2;OD680_Data_3;OD680_Data_4;OD680_Data_5;OD680_Data_6;OD680_Data_7;OD680_Data_8|]

//light data 
let light_Data_1 = 
    let lightColumn =
        light_Data.TryGetColumnObservation<float>("light_treatment_1",Lookup.Exact)
    match lightColumn.HasValue with
    | true -> lightColumn.Value.Value |> Series.observations |> Seq.toArray
    | false -> zeroArray

let light_Data_2 = 
    let lightColumn =
        light_Data.TryGetColumnObservation<float>("light_treatment_2",Lookup.Exact)
    match lightColumn.HasValue with
    | true -> lightColumn.Value.Value |> Series.observations |> Seq.toArray
    | false -> zeroArray

let light_Data_3 = 
    let lightColumn =
        light_Data.TryGetColumnObservation<float>("light_treatment_3",Lookup.Exact)
    match lightColumn.HasValue with
    | true -> lightColumn.Value.Value |> Series.observations |> Seq.toArray
    | false -> zeroArray

let light_Data_4 = 
    let lightColumn =
        light_Data.TryGetColumnObservation<float>("light_treatment_4",Lookup.Exact)
    match lightColumn.HasValue with
    | true -> lightColumn.Value.Value |> Series.observations |> Seq.toArray
    | false -> zeroArray

let light_Data_5 = 
    let lightColumn =
        light_Data.TryGetColumnObservation<float>("light_treatment_5",Lookup.Exact)
    match lightColumn.HasValue with
    | true -> lightColumn.Value.Value |> Series.observations |> Seq.toArray
    | false -> zeroArray

let light_Data_6 = 
    let lightColumn =
        light_Data.TryGetColumnObservation<float>("light_treatment_6",Lookup.Exact)
    match lightColumn.HasValue with
    | true -> lightColumn.Value.Value |> Series.observations |> Seq.toArray
    | false -> zeroArray
        

let light_Data_7 = 
    let lightColumn =
        light_Data.TryGetColumnObservation<float>("light_treatment_7",Lookup.Exact)
    match lightColumn.HasValue with
    | true -> lightColumn.Value.Value |> Series.observations |> Seq.toArray
    | false -> zeroArray

let light_Data_8 = 
    let lightColumn =
        light_Data.TryGetColumnObservation<float>("light_treatment_8",Lookup.Exact)
    match lightColumn.HasValue with
    | true -> lightColumn.Value.Value |> Series.observations |> Seq.toArray
    | false -> zeroArray

let lightData_all = [|light_Data_1;light_Data_2;light_Data_3;light_Data_4;light_Data_5;light_Data_6;light_Data_7;light_Data_8|]



// lightphases -> array with tuple (starttime,endtime)
let lightphase (lightData: (float*float) []) =
    let lightphaseTimeTupleStartEnd : (float * float) array= 
        lightData
        |> Array.groupBy (fun (timeLight,lightIntensity) -> lightIntensity)
        |> Array.filter (fun (constantIntensity,arrayTimeLight) -> arrayTimeLight.Length < 20000 && arrayTimeLight.Length > 300)
        |> Array.map (fun (constantVolume,arrayTimePumpvolume) ->
            (fst (arrayTimePumpvolume.[0]),fst (arrayTimePumpvolume |> Array.last)))
    if lightphaseTimeTupleStartEnd.Length = 3 then 
        [|(fst lightphaseTimeTupleStartEnd.[0], snd lightphaseTimeTupleStartEnd.[1]); (fst lightphaseTimeTupleStartEnd.[2], snd lightphaseTimeTupleStartEnd.[2])|]
    else
        lightphaseTimeTupleStartEnd

// // lightphases -> array with tuple (starttime, endtime)
// let lightphase (lightData: (float * float) []) =
//     // Group by light intensity, filtering for reasonable length constraints
//     let lightphaseTimeTupleStartEnd : (float * float) array = 
//         lightData
//         |> Array.groupBy (fun (timeLight, lightIntensity) -> lightIntensity)
//         |> Array.filter (fun (_, arrayTimeLight) -> arrayTimeLight.Length < 20000 && arrayTimeLight.Length > 300)
//         |> Array.map (fun (_, arrayTimePumpvolume) ->
//             if arrayTimePumpvolume.Length > 0 then 
//                 (fst arrayTimePumpvolume.[0], fst (arrayTimePumpvolume |> Array.last))
//             else 
//                 (0.0, 0.0)) // Fallback tuple in case array is unexpectedly empty

//     // Check the length of the result and return accordingly
//     match lightphaseTimeTupleStartEnd.Length with
//     | 3 -> 
//         [|
//             (fst lightphaseTimeTupleStartEnd.[0], snd lightphaseTimeTupleStartEnd.[1])
//             (fst lightphaseTimeTupleStartEnd.[2], snd lightphaseTimeTupleStartEnd.[2])
//         |]
//     | _ -> lightphaseTimeTupleStartEnd // Return the computed array if it doesn't match length 3

let lightphase_1 =
    (lightphase light_Data_1)

let lightphase_2 =
    (lightphase light_Data_2)

let lightphase_3 =
    (lightphase light_Data_3)

let lightphase_4 =
    (lightphase light_Data_4)

let lightphase_5 =
    (lightphase light_Data_5)

let lightphase_6 =
    (lightphase light_Data_6)

let lightphase_7 =
    (lightphase light_Data_7)

let lightphase_8 =
    (lightphase light_Data_8)

let lightphase_all = [|lightphase_1;lightphase_2;lightphase_3;lightphase_4;lightphase_5;lightphase_6;lightphase_7;lightphase_8|]


In [ ]:
let lightphaseTimeTupleStartEnd : (float * float) array= 
        light_Data_3
        |> Array.groupBy (fun (timeLight,lightIntensity) -> lightIntensity)
        |> Array.filter (fun (constantIntensity,arrayTimeLight) -> arrayTimeLight.Length < 20000 && arrayTimeLight.Length > 300)
        |> Array.map (fun (constantVolume,arrayTimePumpvolume) ->
            (fst (arrayTimePumpvolume.[0]),fst (arrayTimePumpvolume |> Array.last)))

In [ ]:

let sampleNumber odData =
        if odData = OD680_Data_1 then "1"
        else if odData = OD680_Data_2 then "2"
        else if odData = OD680_Data_3 then "3" 
        else if odData = OD680_Data_4 then "4" 
        else if odData = OD680_Data_5 then "5"
        else if odData = OD680_Data_6 then "6"
        else if odData = OD680_Data_7 then "7"  
        else if odData = OD680_Data_8 then "8"
        else "00"

// growphases -> array with tuple (starttime,endtime)
let growphase (pumpData: (float*float) array) (odData: (float*float) array) =
    let growphaseTimeTupleStartEnd = 
        let arrayOfConstantPumpValue =
            pumpData
            |> Array.groupBy (fun (timePump,pumpVolume) -> pumpVolume)
            |> Array.filter (fun (constantVolume,arrayTimePumpvolume) -> arrayTimePumpvolume.Length > 250)
            
        let startEndOfConstantPumpValue : (float * float) array =
            arrayOfConstantPumpValue
            |> Array.map (fun ((constantVolume:float),(arrayTimePumpvolume: (float * float) array)) -> 
                if Array.isEmpty arrayTimePumpvolume then 
                    (0.0,0.0)
                else
                    (float (fst (arrayTimePumpvolume.[0])),float (fst (arrayTimePumpvolume |> Array.last)))
                )
        startEndOfConstantPumpValue
        |> Array.map (fun (startOfgrowphase,endOfGrowphase) -> 
            odData 
            |> Array.filter (fun (time,od680) -> time >= startOfgrowphase && time <= endOfGrowphase && od680 >= (log lowerODCut) && od680 <= (log upperODCut) )) //evtl bei Growphase einbauen damit shapes korrekt sind
            |> Array.map (fun (arrayTimeOD680) ->
                if Array.isEmpty arrayTimeOD680 then 
                    let res = (0.0,0.0)
                    res
                else
                (fst (arrayTimeOD680.[0]), fst (arrayTimeOD680 |> Array.last)))
    growphaseTimeTupleStartEnd

let growphase_1 =
    (growphase pump_Data_1 OD680_Data_1)

let growphase_2 =
    (growphase pump_Data_2 OD680_Data_2)

let growphase_3 =
    (growphase pump_Data_3 OD680_Data_3)

let growphase_4 =
    (growphase pump_Data_4 OD680_Data_4)

let growphase_5 =
    (growphase pump_Data_5 OD680_Data_5)

let growphase_6 =
    (growphase pump_Data_6 OD680_Data_6)

let growphase_7 =
    (growphase pump_Data_7 OD680_Data_7)

let growphase_8 =
    (growphase pump_Data_8 OD680_Data_8)

let growphase_all = [|growphase_1;growphase_2;growphase_3;growphase_4;growphase_5;growphase_6;growphase_7;growphase_8|]


In [ ]:
// linear regression funktion 
let ChartGrowphase_linearFit (growphase : (float * float) array) (odData:(float * float) array) (growphase_index : int) = 
    let arrayGrowphase_Time_OD (growphase : (float * float) array) (odData:(float * float) array) =
        growphase
        |> Array.map (fun (min,max) -> 
            odData 
            |> Array.filter (fun (time,od680) -> time >= min && time <=  max && od680 >= (log lowerODCut) && od680 <= (log upperODCut) )) //evtl bei Growphase einbauen damit shapes korrekt sind

    let growphase_time_OD (growphase_index : int)=
        let growphase_time_OD = (arrayGrowphase_Time_OD growphase odData).[growphase_index]
        growphase_time_OD

    let xs (growphase_index : int) =
        let SeqOfOD680 =
            (growphase_time_OD growphase_index)
            |> Array.map (fun (time,od680) -> time)
        let vector_OD = vector SeqOfOD680
        vector_OD 

    let ys (growphase_index : int)=
        let SeqOfOD680 =
            (growphase_time_OD growphase_index)
            |> Array.map (fun (time,od680) -> od680)
        let vector_OD = vector SeqOfOD680
        vector_OD 
    let fit = 
        if Vector.length (xs growphase_index) > 5 && Vector.length (ys growphase_index) > 5 then
            Fitting.LinearRegression.fit(xs growphase_index ,ys growphase_index,FittingMethod = Fitting.Method.Robust RobustEstimator.TheilSen)
        else
            Fitting.LinearRegression.Coefficients(vector [0.0;0.0])
    let e : float = 
        if Array.isEmpty (growphase_time_OD growphase_index) then
            0.0
        else
            (growphase_time_OD growphase_index)
            |> Array.map (fun (time,od680) -> time)
            |> Array.last
    let s : float =
        if Array.isEmpty (growphase_time_OD growphase_index) then
            0.0
        else
            (growphase_time_OD growphase_index)
            |> Array.map (fun (time,od680) -> time)
            |> Array.sortDescending
            |> Array.last
    let linaer_fit = 
        [s .. 0.1 .. e] 
        |> List.map (fun x -> x,LinearRegression.predict(fit) x)  
        |> Chart.Line 
    linaer_fit

In [ ]:
let arrayGrowphase_Time_OD (growphase : (float * float) array) (odData:(float * float) array) =
    growphase
    |> Array.map (fun (min,max) -> 
        odData 
        |> Array.filter (fun (time,od680) -> time >= min && time <=  max && od680 >= (log lowerODCut) && od680 <= (log upperODCut) )) //evtl bei Growphase einbauen damit shapes korrekt sind

let growphase_time_OD (growphase_index : int)=
    let growphase_time_OD = (arrayGrowphase_Time_OD growphase_1 OD680_Data_1).[growphase_index]
    growphase_time_OD

let xs (growphase_index : int) =
    let SeqOfOD680 =
        (growphase_time_OD growphase_index)
        |> Array.map (fun (time,od680) -> time)
    let vector_OD = vector SeqOfOD680
    vector_OD 

let ys (growphase_index : int)=
    let SeqOfOD680 =
        (growphase_time_OD growphase_index)
        |> Array.map (fun (time,od680) -> od680)
    let vector_OD = vector SeqOfOD680
    vector_OD 
let fit = 
    if Vector.length (xs 0) > 5 && Vector.length (ys 0) > 5 then
        Fitting.LinearRegression.fit(xs 0 ,ys 0,FittingMethod = Fitting.Method.Robust RobustEstimator.TheilSen)
    else
        Fitting.LinearRegression.Coefficients(vector [0.0;0.0])
let e : float = 
    if Array.isEmpty (growphase_time_OD 0) then
        0.0
    else
        (growphase_time_OD 0)
        |> Array.map (fun (time,od680) -> time)
        |> Array.last
let s : float =
    if Array.isEmpty (growphase_time_OD 0) then
        0.0
    else
        (growphase_time_OD 0)
        |> Array.map (fun (time,od680) -> time)
        |> Array.sortDescending
        |> Array.last
let linaer_fit = 
    [s .. 0.1 .. e] 
    |> List.map (fun x -> x,LinearRegression.predict(fit) x)  
    |> Chart.Line 


In [ ]:
// shape list -> array of float tuple (starttime, Endtime)
let shapeGrow (growphase : (float * float) array) (odData : (float * float) array) = 
    growphase
    |> Array.mapi (fun i (start,finish) -> 
        let range = [start..0.1..finish] // because of linear fit so shape matches
        let start = range.[0]
        let finish = (range |> List.last)
        let maxODValue = (snd (odData |> Array.sortBy snd |> Array.last))
        let minODValue = (snd (odData |> Array.sortByDescending snd |> Array.last))
        Shape.init (
            ShapeType = StyleParam.ShapeType.Rectangle,
            X0 = start,
            X1 = finish,
            Y0 = maxODValue + 0.1,
            Y1 = maxODValue + 0.5,
            Opacity = 0.2,
            FillColor = Color.fromHex "#a3e77f",
            Label = ShapeLabel.init(TextTemplate = $"{i+1}", TextAngle = TextAngle.Degrees 0.0, TextPosition = TextPosition.BottomCenter )
    )
    )
    |> Array.toList

In [ ]:
let shapeLight (lightphase : (float * float) array) (odData : (float * float) array) (lightdata : (float * float) array) =  
    let treatment1and2Tuple =
        let groupedByTreatment = 
                lightdata
                |> Array.groupBy (fun (time, lightData) -> lightData)
                |> Array.map (fun (lightTreatment, (time_lightData_array)) -> lightTreatment, ((fst  time_lightData_array.[0]), (fst  (time_lightData_array |> Array.last))))
        groupedByTreatment

    let maxODValue = (snd (odData |> Array.sortBy snd |> Array.last))
    let minODValue = (snd (odData |> Array.sortByDescending snd |> Array.last))
        
    let maxYValue =
        match Array.isEmpty odData with
        | true -> 0.0  // or any default value
        | false -> maxODValue + 0.25

    let minYValue =
        match Array.isEmpty odData with
        | true -> 0.0  // or any default value
        | false -> minODValue - 0.25

    lightphase
    |> Array.mapi (fun i (start,finish) -> 
                Shape.init (
                    ShapeType = StyleParam.ShapeType.Rectangle,
                    X0 = start,
                    X1 = finish,
                    Y0 = maxYValue,
                    Y1 = minYValue,
                    Opacity = 0.1,
                    FillColor = Color.fromHex "#ffffba",
                    Label = ShapeLabel.init(TextTemplate = $"Highlight {fst treatment1and2Tuple.[i]}", TextAngle = TextAngle.Degrees 0.0, TextPosition = TextPosition.BottomCenter )
                    )
    )
    |> Array.toList

In [ ]:

// pump-graph combined with OD graph and linear fit for growphase
let endGraph (growphase : (float * float) array) (odData:(float * float) array) (pump_Data : (float * float) array) (lightData : (float * float) array)=
    let linearRegression_all_chartLine =
        [
            for i in 0 .. growphase.Length - 1 do
            (ChartGrowphase_linearFit growphase odData i) |> Chart.withTraceInfo $"Robust TheilSen {i}"
            |> Chart.withLineStyle(Width = 2.25, Color = Color.fromHex("#fb2e01"), Dash=StyleParam.DrawingStyle.Dot)
            |> Chart.withAxisAnchor(X=1,Y=1)
            |> Chart.withTraceInfo(Name = $"Phase ID: {i}")
        ]
        |> Chart.combine
    let odData_all_chartPoint =
        Chart.Point(xy = (odData))
        |> Chart.withLineStyle(Color =Color.fromHex("#7bc043"))
        |> Chart.withTraceInfo(Name = "Ln OD680")
        |> Chart.withAxisAnchor(X=1,Y=1)
    let pumpData_all_chartPoint = 
        Chart.Point(xy = (pump_Data))
        |> Chart.withLineStyle(Color =Color.fromHex("#005b96"))
        |> Chart.withTraceInfo(Name = "pumpvolume (ml)")
        |> Chart.withAxisAnchor(X=1,Y=2)
    let lightData_all_chartPoint = 
        Chart.Point(xy = (lightData))
        |> Chart.withLineStyle(Color =Color.fromHex("#ffffba"))
        |> Chart.withTraceInfo(Name = "Lighttreatment (µE)")
        |> Chart.withAxisAnchor(X=1,Y=2)
    [
        odData_all_chartPoint
        linearRegression_all_chartLine
        lightData_all_chartPoint
        pumpData_all_chartPoint
    ]
    |> Chart.combine
    |> Chart.withTemplate ChartTemplates.lightMirrored
    |> Chart.withXAxisStyle("time (h)",
        Id=StyleParam.SubPlotId.XAxis 1,
        MinMax= (0.0, (fst (odData |> Array.last)) + 1.0),
        Side=StyleParam.Side.Bottom,
        ShowLine=true,
        LineColor=Color.fromKeyword Black
        )
    |> Chart.withYAxisStyle(
        $"Ln OD680 cylinder {sampleNumber odData}",
        Side=StyleParam.Side.Left,
        MinMax= (((snd (odData |> Array.sortByDescending snd |> Array.last)) - 0.25),((snd (odData |> Array.sortBy snd |> Array.last)) + 0.2)),
        Id=StyleParam.SubPlotId.YAxis 1,
        ShowLine=true,
        LineColor=Color.fromKeyword Black,
        ZeroLine = false
        )
    |> Chart.withYAxisStyle(
        $"pumpvolume (ml) and light treatment (µE) cylinder {sampleNumber odData}", 
        Side=StyleParam.Side.Right,
        MinMax= (((snd (lightData |> Array.sortByDescending snd |> Array.last)) - 0.1),((snd (lightData |> Array.sortBy snd |> Array.last)) * 1.5)),
        Id=StyleParam.SubPlotId.YAxis 2,
        ShowLine=true,
        LineColor= Color.fromKeyword(Black),
        ZeroLine = false
        )

// pump-graph combined with OD graph and linear fit for growphase
let endGraph_single (growphase : (float * float) array) (lightphase : (float * float) array) (odData:(float * float) array) (pump_Data : (float * float) array) (lightData : (float * float) array) =
    let linearRegression_all_chartLine =
        [
            for i in 0 .. growphase.Length - 1 do
            (ChartGrowphase_linearFit growphase odData i) |> Chart.withTraceInfo $"Robust TheilSen {i}"
            |> Chart.withLineStyle(Width = 2.25, Color = Color.fromHex("#fb2e01"), Dash=StyleParam.DrawingStyle.Dot)
            |> Chart.withAxisAnchor(Y=1)
            |> Chart.withTraceInfo(Name = $"Phase ID: {i}")
        ]
        |> Chart.combine
    let odData_all_chartPoint =
        Chart.Point(xy = (odData))
        |> Chart.withLineStyle(Color =Color.fromHex("#7bc043"))
        |> Chart.withTraceInfo(Name = "Ln OD680")
        |> Chart.withAxisAnchor(Y=1)
    let pumpData_all_chartPoint = 
        Chart.Point(xy = (pump_Data))
        |> Chart.withLineStyle(Color =Color.fromHex("#005b96"))
        |> Chart.withTraceInfo(Name = "pumpvolume (ml)")
        |> Chart.withAxisAnchor(Y=2)
    let lightData_all_chartPoint = 
        Chart.Point(xy = (lightData))
        |> Chart.withLineStyle(Color =Color.fromHex("#ffffba"))
        |> Chart.withTraceInfo(Name = "Lighttreatment (µE)")
        |> Chart.withAxisAnchor(Y=2)
    [
        odData_all_chartPoint
        linearRegression_all_chartLine
        lightData_all_chartPoint
        pumpData_all_chartPoint
    ]
    |> Chart.combine
    |> Chart.withShapes(shapes = (shapeGrow growphase odData), Append = true)
    |> Chart.withShapes(shapes = (shapeLight lightphase odData lightData), Append = true)
    |> Chart.withTemplate ChartTemplates.lightMirrored
    |> Chart.withLegendStyle(Orientation = StyleParam.Orientation.Horizontal)
    |> Chart.withSize(Width = 1600, Height = 800)
    |> Chart.withTitle($"OD680 with linear regression of the growphases of cylinder {sampleNumber odData} with pumpdata")
    |> Chart.withXAxisStyle("time (h)",
        Id=StyleParam.SubPlotId.XAxis 1,
        MinMax= (0.0, (fst (odData |> Array.last)) + 1.0),
        Side=StyleParam.Side.Bottom,
        ShowLine=true,
        LineColor=Color.fromKeyword Black,
        ZeroLine = false
        )
    |> Chart.withYAxisStyle(
        $"Ln OD680 cylinder {sampleNumber odData}",
        Side=StyleParam.Side.Left,
        MinMax= (((snd (odData |> Array.sortByDescending snd |> Array.last)) - 0.25),((snd (odData |> Array.sortBy snd |> Array.last)) + 0.2)),
        Id=StyleParam.SubPlotId.YAxis 1,
        Overlaying= StyleParam.LinearAxisId.Y 2,
        ShowLine=true,
        LineColor=Color.fromKeyword Black,
        ZeroLine = false
        )
    |> Chart.withYAxisStyle(
        $"pumpvolume (ml) and light treatment (µE) cylinder {sampleNumber odData}", 
        Side=StyleParam.Side.Right,
        MinMax= (((snd (pump_Data |> Array.sortByDescending snd |> Array.last)) * 0.8),((snd (lightData |> Array.sortBy snd |> Array.last)) * 1.05)),
        Id=StyleParam.SubPlotId.YAxis 2,
        ShowLine=false,
        LineColor= Color.fromKeyword(Black),
        ZeroLine = false
        )

In [ ]:
((snd (OD680_Data_1 |> Array.sortByDescending snd |> Array.last)) - 0.25),((snd (OD680_Data_1 |> Array.sortBy snd |> Array.last)) + 0.2)

Item1,-3.3048455380464974
Item2,5.190432586778736


In [ ]:
// Chart.Point(OD680_Data_1)
// |> Chart.withLineStyle(Color =Color.fromHex("#7bc043"))
// |> Chart.show

In [ ]:
// growphase_1, OD680_Data_1, pump_Data_1, light_Data_1

In [ ]:
let firsttest = (endGraph_single growphase_1 lightphase_1 OD680_Data_1 pump_Data_all.[0] light_Data_1)


firsttest
|> Chart.withShapes(shapes = (shapeGrow growphase_1 OD680_Data_1), Append = true)
|> Chart.withShapes(shapes = (shapeLight lightphase_1 OD680_Data_1 light_Data_1), Append = true)
|> Chart.withSize(Width= 1700, Height= 900)
|> Chart.show

In [ ]:
endGraph growphase_1 OD680_Data_1 pump_Data_1 light_Data_1
// |> Chart.withShapes(shapes = (shapeGrow growphase_1 OD680_Data_1), Append = true)
// |> Chart.withShapes(shapes = (shapeLight lightphase_1 OD680_Data_1 light_Data_1), Append = true)
|> Chart.withSize(Width= 1700, Height= 900)
|> Chart.show


In [ ]:

// slope/growratefunction for individual growphase of one cylinder
let slopeOfGrowphase_X (growphase : (float * float) array) (odData:(float * float) array) (growphase_index : int) = 
    let arrayGrowphase_Time_OD (growphase : (float * float) array) (odData:(float * float) array) =
        growphase
        |> Array.map (fun (min,max) -> 
            odData 
            |> Array.filter (fun (time,od680) -> time > min && time < max && od680 > (log lowerODCut) && od680 < (log upperODCut)))

    let growphase_time_OD (growphase_index : int)=
        let growphase_time_OD = (arrayGrowphase_Time_OD growphase odData).[growphase_index]
        growphase_time_OD

    let xs (growphase_index : int) =
        let SeqOfOD680 =
            (growphase_time_OD growphase_index)
            |> Array.map (fun (time,od680) -> time)
        let vector_OD = vector SeqOfOD680
        vector_OD 

    let ys (growphase_index : int)=
        let SeqOfOD680 =
            (growphase_time_OD growphase_index)
            |> Array.map (fun (time,od680) -> od680)
        let vector_OD = vector SeqOfOD680
        vector_OD 
    let fit = 
        if Vector.length (xs growphase_index) > 5 && Vector.length (ys growphase_index) > 5 then
            Fitting.LinearRegression.fit(xs growphase_index ,ys growphase_index,FittingMethod = Fitting.Method.Robust RobustEstimator.TheilSen)
        else
            Fitting.LinearRegression.Coefficients(vector [0.0;0.0])
    fit.Coefficients.[1]


In [ ]:

// function for table values -> List List string
let table_list (growphase : (float * float) array) (odData:(float * float) array) : list<Library.tableRow> = 
    let growrateList =
        [
            for i in 0 .. growphase.Length - 1 do
            (slopeOfGrowphase_X growphase odData i)
        ]
        |> List.toArray
    let list_int = 
        [|1 .. growphase.Length|]
    let start = 
        growphase
        |> Array.map (fun (start,finish) -> start)
    let finish = 
        growphase
        |> Array.map (fun (start,finish) -> finish)
    let rowItemsList : Library.tableRow list =
        [for i in 0 .. growphase.Length - 1 do
            let list_int_index: int = list_int.[i]
            let start_index: float = start.[i]
            let finish_index: float = finish.[i]
            let growrate_index: float = growrateList.[i]
            let duplicationTime: float = (log(2.0) /(growrate_index))
            {
                Library.tableRow.PhaseID = list_int_index;
                Library.tableRow.startTimeGrowphase = start_index;
                Library.tableRow.endTimeGrowphase = finish_index;
                Library.tableRow.slopeOrGrowrateOfLinearRegressionOrGrowphase = growrate_index;
                Library.tableRow.duplicationTimeOfGrowphase = duplicationTime
            }
        ]
    rowItemsList
printfn("Bis Hier3")

Bis Hier3


In [ ]:
// function for boxblot of slopes of one cylinder
let boxplot_slope (growphase : (float * float) array) (odData:(float * float) array) = 
    let y = 
        table_list (growphase) (odData) |> List.map (fun x -> float x.slopeOrGrowrateOfLinearRegressionOrGrowphase)
    Chart.BoxPlot(Y = y, Name = "growrate (h<sup>-1</sub>)",Jitter=0.1,BoxPoints=StyleParam.BoxPoints.SuspectedOutliers,BoxMean=StyleParam.BoxMean.True, OutlineWidth = 1.5)
    |> Chart.withLineStyle(Color = Color.fromHex("#005b96"))

// function for pointchart of slopes of one cylinder
let pointchart_slope (growphase : (float * float) array) (odData:(float * float) array) = 
    let xy = 
        table_list (growphase) (odData) |> Seq.map (fun x -> (float x.PhaseID, float x.slopeOrGrowrateOfLinearRegressionOrGrowphase)) 
    Chart.Point(xy = xy, Name = "growrate (h<sup>-1</sub>)")
    |> Chart.withLineStyle(Color = Color.fromHex("#005b96"))

let sortSlopeList (growphase : (float * float) array) (odData:(float * float) array) = 
    let slopelist = 
        (table_list growphase odData)
        |> List.map (fun rowItem -> (rowItem.slopeOrGrowrateOfLinearRegressionOrGrowphase))
    slopelist
    |> List.sort
printfn("Bis Hier4")

Bis Hier4


In [ ]:
// function for table
let table (growphase : (float * float) array) (odData:(float * float) array) = 
    let header : seq<string> = seq [ "<b>Phase ID</b>"; "starttime of growphase (h)"; "endtime of growphase (h)"; "growrate (h<sup>-1</sub>)"; "duplicationtime (Td) (h)" ]
    let rows : seq<seq<float>> = 
        table_list growphase odData
        |> Seq.map (fun rowItem ->
                seq [
                    float rowItem.PhaseID;
                    round 2 rowItem.startTimeGrowphase;
                    round 2 rowItem.endTimeGrowphase;
                    round 3 rowItem.slopeOrGrowrateOfLinearRegressionOrGrowphase;
                    round 3 rowItem.duplicationTimeOfGrowphase
                    ]
            )
    Chart.Table(
        header,
        rows,
        HeaderAlign = StyleParam.HorizontalAlign.Left,
        CellsAlign = StyleParam.HorizontalAlign.Left,
        MultiColumnWidth = [1.5;2.5;2.5;4.0],
        HeaderFillColor = Color.fromString( "DarkGray" ),
        HeaderHeight = 2,
        HeaderOutlineColor = Color.fromString( "Grey" ),
        HeaderOutlineWidth = 2.0,
        CellsFillColor = Color.fromString( "LightGray" ),
        CellsOutlineColor = Color.fromString( "Grey" ),
        CellsOutlineWidth = 1.5,
        CellsHeight = 25
        )
    |> Chart.withSize(Width=1600,Height=800)

// function for table
// let description (growphase : (float * float) array) (odData:(float * float) array) = 
//     // some styling for a html table
//     let style = "<style>table {font-family: arial, sans-serif;border-collapse: collapse;width: 80%;height: 100%}td, th {border: 2px solid #dddddd;text-align: left;padding: 8px;}tr:nth-child(even) {background-color: #dddddd;}</style>"
//     // header row of the table
//     let header = "<tr><th>Phase ID</th><th>starttime of growphase (h)</th><th>endtime of growphase (h)</th><th>growrate (h<sup>-1</sup>)</th><th>duplicationtime (h)</th></tr>"
//     // table rows
//     let rows = 
//         (table_list growphase odData)
//         |> List.map (fun rowItem ->
//                 // create a table row with phase id, the start and end of the treatment formatted as a float with one significant figure (defined by %.1f) and the slope with four significant figures.
//                 // the slope is stored within the fit coefficients as [intersect;slope]
//             $"<tr><td>{rowItem.PhaseID}</td><td>%.2f{(rowItem.startTimeGrowphase)}</td><td>%.2f{(rowItem.endTimeGrowphase)}</td><td>%.4f{(rowItem.slopeOrGrowrateOfLinearRegressionOrGrowphase)}</td><td>%.4f{(rowItem.duplicationTimeOfGrowphase)}</td></tr>"
//         )
//     // constructed table
//     let table = $"{style}<table>{header}{rows}</table>"
//     // convert the table string to a giraffe node element to be compatible with Plotly.NET
//     Giraffe.ViewEngine.HtmlElements.rawText table


In [ ]:
// analysis function 
let analysisFull (growphase : (float * float) array) (lightphase : (float * float) array) (odData:(float * float) array) (pump_Data :(float * float) array) (lightData : (float * float) array) =

    let subplotGrid = 
        [|
            [| StyleParam.LinearAxisId.X 1 , StyleParam.LinearAxisId.Y 1 ; StyleParam.LinearAxisId.X 2 , StyleParam.LinearAxisId.Y 2 |]
            [| StyleParam.LinearAxisId.X 3 , StyleParam.LinearAxisId.Y 3 ; StyleParam.LinearAxisId.X 4 , StyleParam.LinearAxisId.Y 4 |]
        |]

    let pumpData_all_chartPoint = 
        Chart.Point(xy = (pump_Data))
        |> Chart.withLineStyle(Color =Color.fromHex("#005b96"))
        |> Chart.withTraceInfo(Name = "pumpvolume (ml)")

    [
        
        (endGraph (growphase) (odData) (pump_Data) (lightData))
        |> Chart.withTemplate ChartTemplates.lightMirrored
        |> Chart.withAxisAnchor(X=1,Y=1);

        //(endGraph (growphase) (odData) (pump_Data))
        pumpData_all_chartPoint
        |> Chart.withTemplate ChartTemplates.lightMirrored
        //|> Chart.withLayoutStyle(PlotBGColor = Color.fromARGB 0 0 0 0, PaperBGColor = Color.fromARGB 0 0 0 0)
        |> Chart.withAxisAnchor(X=1,Y=2);
    
        (pointchart_slope (growphase) (odData))
        |> Chart.withTemplate ChartTemplates.lightMirrored
        |> Chart.withAxisAnchor(X=3,Y=3);

        (boxplot_slope (growphase) (odData))
        |> Chart.withTemplate ChartTemplates.lightMirrored
        |> Chart.withAxisAnchor(X=4,Y=4);

        table growphase odData
        |> Chart.withTemplate ChartTemplates.lightMirrored
        //|> Chart.withAxisAnchor(X=5,Y=5);

    ]
    |> Chart.Grid(nRows=3, nCols=2, Pattern = StyleParam.LayoutGridPattern.Independent)
    |> Chart.withTemplate ChartTemplates.lightMirrored
    |> Chart.withShapes(shapes = (shapeGrow growphase odData), Append = true)
    |> Chart.withShapes(shapes = (shapeLight lightphase odData lightData), Append = true)
    |> Chart.withLegendStyle(Orientation = StyleParam.Orientation.Horizontal)
    |> Chart.withXAxisStyle("",
        Id=StyleParam.SubPlotId.XAxis 1,
        MinMax= (0.0, (fst (odData |> Array.last)) + 1.0),
        Domain = (0.00, 1.00),
        Side=StyleParam.Side.Bottom,
        ShowLine=true,
        LineColor=Color.fromKeyword Black
        )
    |> Chart.withYAxisStyle(
        $"Ln OD680 cylinder {sampleNumber odData}",
        Side=StyleParam.Side.TopLeft,
        MinMax= (((snd (odData |> Array.sortByDescending snd |> Array.last)) - 0.25 ),((snd (odData |> Array.sortBy snd |> Array.last)) + 0.25)),
        Id=StyleParam.SubPlotId.YAxis 1,
        Domain = (0.725, 1.00),
        ShowLine=true,
        LineColor=Color.fromKeyword Black
        )
    |> Chart.withYAxisStyle(
        $"pumpvolume (ml) cylinder {sampleNumber odData}", 
        Side=StyleParam.Side.Right,
        MinMax= (((snd (pump_Data |> Array.sortByDescending snd |> Array.last)) - 0.05),((snd (pump_Data |> Array.sortBy snd |> Array.last)) * 1.)),
        Id=StyleParam.SubPlotId.YAxis 2,
        //Overlaying=StyleParam.LinearAxisId.Y 1,
        Domain = (0.625, 0.725),
        ShowLine=true,
        LineColor=Color.fromKeyword Black
        )
    |> Chart.withXAxisStyle("time (h)",
        Id=StyleParam.SubPlotId.XAxis 2,
        MinMax= (0.0, (fst (odData |> Array.last)) + 1.0),
        Domain = (0.00, 1.00),
        Side=StyleParam.Side.Bottom,
        ShowLine=true,
        LineColor=Color.fromKeyword Black
        )
    |> Chart.withXAxisStyle($"growphases in cylinder {sampleNumber odData}",
        Id=StyleParam.SubPlotId.XAxis 3,
        MinMax= (0.0, growphase.Length + 1),
        Domain = (0.00, 0.49),
        Side=StyleParam.Side.Bottom,
        ShowLine=true,
        LineColor=Color.fromKeyword Black
        )
    |> Chart.withYAxisStyle("growrate (h<sup>-1</sub>)",
        Id=StyleParam.SubPlotId.YAxis 3,
        MinMax= (((sortSlopeList growphase odData).[0]) - 0.005, ((sortSlopeList growphase odData) |> List.last) + 0.005),
        Domain = (0.325, 0.55),
        Side=StyleParam.Side.Left,
        ShowLine=true,
        LineColor=Color.fromKeyword Black
        )
    |> Chart.withXAxisStyle($"cylinder {sampleNumber odData}",
        Id=StyleParam.SubPlotId.XAxis 4,
        Domain = (0.51, 1.00),
        Side=StyleParam.Side.Bottom,
        ShowLine=true,
        LineColor=Color.fromKeyword Black
        )      
    |> Chart.withYAxisStyle("growrate (h<sup>-1</sub>)",
        Id=StyleParam.SubPlotId.YAxis 4,
        MinMax= (((sortSlopeList growphase odData).[0]) - 0.005, ((sortSlopeList growphase odData) |> List.last) + 0.005),
        Domain = (0.325, 0.55),
        Side=StyleParam.Side.Right,
        ShowLine=true,
        LineColor=Color.fromKeyword Black
        )
    |> Chart.withXAxisStyle("",
        Id=StyleParam.SubPlotId.XAxis 5,
        Domain = (0.00, 0.33),
        Side=StyleParam.Side.Bottom,
        ShowLine=true,
        LineColor=Color.fromKeyword Black
        )
    |> Chart.withYAxisStyle(
        "",
        Side=StyleParam.Side.TopLeft,
        Id=StyleParam.SubPlotId.YAxis 5,
        Domain = (0.00, 1.00),
        ShowLine=true,
        LineColor=Color.fromKeyword Black
        )
    |> Chart.withLayoutGridStyle (YGap = 0.3, XGap = 0.1, SubPlots = subplotGrid)
    |> Chart.withLayoutStyle(ShowLegend = false)
    |> Chart.withLayoutStyle(Font=(Font.init(Family= StyleParam.FontFamily.Arial,Size=14)))
    |> Chart.withSize (1500, 1600)
    |> Chart.withConfig (Config.init (ToImageButtonOptions = ConfigObjects.ToImageButtonOptions.init(Format = StyleParam.ImageFormat.SVG)))
    //|> Chart.withDescription [description growphase odData]
    |> Chart.withTitle($"growphases analysis of cylinder {sampleNumber odData}")

In [ ]:
analysisFull growphase_1 lightphase_1 OD680_Data_1 pump_Data_1 light_Data_1
|> Chart.show

In [ ]:

let tableExport (growphase : (float * float) array) (odData:(float * float) array) =
    FileIO.writeToFile true (currentProjectPath + $"{seperatorAsString}Output{seperatorAsString}" + $"tableOfCylinder_{sampleNumber odData}" + ".csv") ( (table_list growphase odData) |> Seq.map (fun rowItem -> $"{rowItem.PhaseID};%.2f{(rowItem.startTimeGrowphase)};%.2f{(rowItem.endTimeGrowphase)};%.4f{(rowItem.slopeOrGrowrateOfLinearRegressionOrGrowphase)};%.4f{(rowItem.duplicationTimeOfGrowphase)}"))
let tableshow (growphase : (float * float) array) (odData:(float * float) array) =
    table growphase odData
    |> Chart.withTitle($"table: growphase characteristics of cylinder {sampleNumber odData}")

In [ ]:
growphase_all

index value 0 index value 0 (0.083333, 19) Item1 0.083333 Item2 19 1 index value 0 (0.083333, 19) Item1 0.083333 Item2 19 2 index value 0 (0.083333, 19) Item1 0.083333 Item2 19 3 index value 0 (0.083333, 19) Item1 0.083333 Item2 19 4 index value 0 (0.083333, 19) Item1 0.083333 Item2 19 5 index value 0 (0.083333, 19) Item1 0.083333 Item2 19 6 index value 0 (0.083333, 19) Item1 0.083333 Item2 19 7 index value 0 (0.083333, 19) Item1 0.083333 Item2 19

In [ ]:
lightphase_all

index value 0 index value 0 (0, 19) Item1 0 Item2 19 1 index value 0 (0, 19) Item1 0 Item2 19 2 index value 0 (0, 19) Item1 0 Item2 19 3 index value 0 (0, 19) Item1 0 Item2 19 4 index value 0 (0, 19) Item1 0 Item2 19 5 index value 0 (0, 19) Item1 0 Item2 19 6 index value 0 (0, 19) Item1 0 Item2 19 7 index value 0 (0, 19) Item1 0 Item2 19